# Curating a genome scale model (second pass-draft)

This notebook has been tested on [jprime.lbl.gov](jprime.lbl.gov) with the biodesign_3.7 kernel.

It starts with the model that gets output by the annotation_gr.ipynb notebook.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json
import urllib
import cobra
import cplex
import os
import requests
import collections

# Getting and preparing the genome-scale model

## Load *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("GSMs/Ropacus_curation_first_pass.xml")
model

Name,ropacus_curated_first_pass
Memory address,0x07f2f55b74f50
Number of metabolites,1581
Number of reactions,2380
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


## Starting MEMOTE Output

In [3]:
IFrame('memotes/ropacus_carveme_grampos.htm', 1500, 800)

# Basic Function Definitions

In [8]:
def should_be_balanced(r):
    if r.id.startswith('EX_') or r.id.startswith('sink_') or r.id.startswith('Growth'):
        return False
    else:
        return True
    
def has_metabolite_with_multiple_formulas(r):
    for m in r.metabolites:
        if len(m.formula.split(';')) > 1:
            return True
    return False

# Status Report

In [4]:
for i in range(1,5):
    num_formulas = [m for m in model.metabolites if len(m.formula.split(';')) == i]
    print(f'{len(num_formulas)} of {len(model.metabolites)} metabolites have {i} formula(s)')

1513 of 1581 metabolites have 1 formula(s)
63 of 1581 metabolites have 2 formula(s)
5 of 1581 metabolites have 3 formula(s)
0 of 1581 metabolites have 4 formula(s)


In [6]:
unbalanced = [r for r in model.reactions if should_be_balanced(r) and r.check_mass_balance() != {}]
unbalanced_but_okay = [r for r in model.reactions if not should_be_balanced(r) and r.check_mass_balance() != {}]
balanced = [r for r in model.reactions if r.check_mass_balance() == {}]

print(f'{len(unbalanced)} of the {len(model.reactions)} reactions in the model are wrongly unbalanced')
print(f'{len(unbalanced_but_okay)} of the {len(model.reactions)} reactions in the model are properly unbalanced')
print(f'{len(balanced)} of the {len(model.reactions)} reactions in the model are balanced')

166 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
1986 of the 2380 reactions in the model are balanced


In [9]:
unbalanced_multiple_formulas = [r for r in unbalanced if has_metabolite_with_multiple_formulas(r)]
unbalanced_but_okay_multiple_formulas = [r for r in unbalanced_but_okay if has_metabolite_with_multiple_formulas(r)]
balanced_multiple_formulas   = [r for r in   balanced if has_metabolite_with_multiple_formulas(r)]

print(f'{len(unbalanced_multiple_formulas)} of the {len(unbalanced)} improperly unbalanced reactions in the model have at least one metabolite with multiple formulas')
print(f'{len(unbalanced_but_okay_multiple_formulas)} of the {len(unbalanced_but_okay)} properly unbalanced reactions in the model have at least one metabolite with multiple formulas')
print(f'{len(balanced_multiple_formulas)} of the {len(balanced)} balanced reactions in the model have at least one metabolite with multiple formulas')

163 of the 166 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
14 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
26 of the 1986 balanced reactions in the model have at least one metabolite with multiple formulas


In [19]:
multiple_formula_metabolites = [m for m in model.metabolites if len(m.formula.split(';')) > 1]
sorted_multiple_formula_metabolites = sorted(multiple_formula_metabolites, key=lambda m: len(m.reactions))
    
sorted_multiple_formula_metabolites[0]

frozenset({<Reaction ALLTNt2rpp at 0x7f300b585b10>})

Define function to give information on a given metabolite to help solve its formula

In [30]:
def get_metabolite_info(m):
    print(m.id, m.name, m.formula)
    for r in m.reactions:
        print(r.name)
        print(r.reaction)
        print(r.check_mass_balance())
        for m in r.metabolites:
            print(m.id, m.formula)
        print()

In [31]:
get_metabolite_info(sorted_multiple_formula_metabolites[0])

alltn_p Allantoin C4H5N4O3;C4H6N4O3
Allantoin transport in via proton symport (periplasm)
alltn_p + h_p <=> alltn_c + h_c
{}
alltn_p C4H5N4O3;C4H6N4O3
h_p H
alltn_c C4H5N4O3;C4H6N4O3
h_c H



Not much we can do with this

In [32]:
get_metabolite_info(sorted_multiple_formula_metabolites[1])

gsn_p Guanosine C10H13N5O5;C10H12N5O5
Guanosine transport in via proton symport (periplasm)
gsn_p + h_p --> gsn_c + h_c
{}
gsn_p C10H13N5O5;C10H12N5O5
h_p H
gsn_c C10H13N5O5;C10H12N5O5
h_c H

5'-nucleotidase (GMP)
gmp_p + h2o_p --> gsn_p + pi_p
{'C': 10.0, 'H': 12.0, 'N': 5.0, 'O': 5.0}
gmp_p C10H12N5O8P
h2o_p H2O
gsn_p C10H13N5O5;C10H12N5O5
pi_p HPO4



Since the mass error is C10H12N5O5. This is evidence that the formula for gmp_c and gmp_p is not C10H12N5O5

In [50]:
model.metabolites.get_by_id('gsn_p').formula = 'C10H13N5O5'
model.metabolites.get_by_id('gsn_c').formula = 'C10H13N5O5'

In [51]:
get_metabolite_info(sorted_multiple_formula_metabolites[1])

gsn_p Guanosine C10H13N5O5
Guanosine transport in via proton symport (periplasm)
gsn_p + h_p --> gsn_c + h_c
{}
gsn_p C10H13N5O5
h_p H
gsn_c C10H13N5O5
h_c H

5'-nucleotidase (GMP)
gmp_p + h2o_p --> gsn_p + pi_p
{}
gmp_p C10H12N5O8P
h2o_p H2O
gsn_p C10H13N5O5
pi_p HPO4



Now both reactions are balanced

In [33]:
get_metabolite_info(sorted_multiple_formula_metabolites[2])

4izp_c 4-Imidazolone-5-propanoate C6H6N2O3;C6H7N2O3
Urocanase
h2o_c + urcan_c --> 4izp_c
{'H': 6.0, 'O': 3.0, 'C': 6.0, 'N': 2.0}
h2o_c H2O
urcan_c C6H5N2O2
4izp_c C6H6N2O3;C6H7N2O3

Imidazolonepropionase
4izp_c + h2o_c --> forglu_c + h_c
{'H': 3.0, 'O': 1.0}
h2o_c H2O
4izp_c C6H6N2O3;C6H7N2O3
h_c H
forglu_c C6H9N2O4;C6H8N2O4



In [52]:
model.metabolites.get_by_id('4izp_c').formula = 'C6H7N2O3'

In [53]:
get_metabolite_info(sorted_multiple_formula_metabolites[2])

4izp_c 4-Imidazolone-5-propanoate C6H7N2O3
Urocanase
h2o_c + urcan_c --> 4izp_c
{}
h2o_c H2O
urcan_c C6H5N2O2
4izp_c C6H7N2O3

Imidazolonepropionase
4izp_c + h2o_c --> forglu_c + h_c
{'H': 9.0, 'O': 4.0, 'C': 6.0, 'N': 2.0}
h2o_c H2O
4izp_c C6H7N2O3
h_c H
forglu_c C6H9N2O4;C6H8N2O4



In [54]:
model.metabolites.get_by_id('forglu_c').formula = 'C6H8N2O4'

In [55]:
get_metabolite_info(sorted_multiple_formula_metabolites[2])

4izp_c 4-Imidazolone-5-propanoate C6H7N2O3
Urocanase
h2o_c + urcan_c --> 4izp_c
{}
h2o_c H2O
urcan_c C6H5N2O2
4izp_c C6H7N2O3

Imidazolonepropionase
4izp_c + h2o_c --> forglu_c + h_c
{}
h2o_c H2O
4izp_c C6H7N2O3
h_c H
forglu_c C6H8N2O4



Now these reactions are balanced

In [34]:
get_metabolite_info(sorted_multiple_formula_metabolites[3])

pan4p_c Pantetheine 4'-phosphate C11H20N2O7PS;C11H21N2O7PS
Phosphopantothenoylcysteine decarboxylase
4ppcys_c + h_c --> co2_c + pan4p_c
{'C': -1.0, 'H': 1.0, 'O': -2.0}
4ppcys_c C12H20N2O9PS;C12H19N2O9PS
h_c H
co2_c CO2
pan4p_c C11H20N2O7PS;C11H21N2O7PS

Pantetheine-phosphate adenylyltransferase
atp_c + h_c + pan4p_c --> dpcoa_c + ppi_c
{'C': -11.0, 'H': -20.0, 'N': -2.0, 'O': -7.0, 'P': -1.0, 'S': -1.0}
atp_c C10H12N5O13P3
h_c H
pan4p_c C11H20N2O7PS;C11H21N2O7PS
dpcoa_c C21H33N7O13P2S
ppi_c HO7P2



In [56]:
model.metabolites.get_by_id('pan4p_c').formula = 'C11H21N2O7PS'

In [57]:
get_metabolite_info(sorted_multiple_formula_metabolites[3])

pan4p_c Pantetheine 4'-phosphate C11H21N2O7PS
Phosphopantothenoylcysteine decarboxylase
4ppcys_c + h_c --> co2_c + pan4p_c
{'C': -12.0, 'H': -19.0, 'N': -2.0, 'O': -9.0, 'P': -1.0, 'S': -1.0}
4ppcys_c C12H20N2O9PS;C12H19N2O9PS
h_c H
co2_c CO2
pan4p_c C11H21N2O7PS

Pantetheine-phosphate adenylyltransferase
atp_c + h_c + pan4p_c --> dpcoa_c + ppi_c
{}
atp_c C10H12N5O13P3
h_c H
pan4p_c C11H21N2O7PS
dpcoa_c C21H33N7O13P2S
ppi_c HO7P2



In [58]:
model.metabolites.get_by_id('4ppcys_c').formula = 'C12H20N2O9PS'

In [59]:
get_metabolite_info(sorted_multiple_formula_metabolites[3])

pan4p_c Pantetheine 4'-phosphate C11H21N2O7PS
Phosphopantothenoylcysteine decarboxylase
4ppcys_c + h_c --> co2_c + pan4p_c
{}
4ppcys_c C12H20N2O9PS
h_c H
co2_c CO2
pan4p_c C11H21N2O7PS

Pantetheine-phosphate adenylyltransferase
atp_c + h_c + pan4p_c --> dpcoa_c + ppi_c
{}
atp_c C10H12N5O13P3
h_c H
pan4p_c C11H21N2O7PS
dpcoa_c C21H33N7O13P2S
ppi_c HO7P2



Now these reactions are balanced

In [60]:
get_metabolite_info(sorted_multiple_formula_metabolites[4])

ad_e Acetamide C2H6NO;C2H5NO
R_EX_ad_e
ad_e --> 
{'C': -4.0, 'H': -11.0, 'N': -2.0, 'O': -2.0}
ad_e C2H6NO;C2H5NO

ADtr
ad_e <=> ad_c
{}
ad_e C2H6NO;C2H5NO
ad_c C2H6NO;C2H5NO



Can't do much with this info

In [61]:
get_metabolite_info(sorted_multiple_formula_metabolites[5])

ad_c Acetamide C2H6NO;C2H5NO
AMID4
ad_c + h2o_c --> ac_c + nh4_c
{'H': -6.0, 'O': -1.0, 'C': -2.0, 'N': -1.0}
h2o_c H2O
ad_c C2H6NO;C2H5NO
ac_c C2H3O2
nh4_c H4N

ADtr
ad_e <=> ad_c
{}
ad_e C2H6NO;C2H5NO
ad_c C2H6NO;C2H5NO



In [62]:
model.metabolites.get_by_id('ad_c').formula = 'C2H5NO'
model.metabolites.get_by_id('ad_e').formula = 'C2H5NO'

In [63]:
get_metabolite_info(sorted_multiple_formula_metabolites[5])

ad_c Acetamide C2H5NO
AMID4
ad_c + h2o_c --> ac_c + nh4_c
{}
h2o_c H2O
ad_c C2H5NO
ac_c C2H3O2
nh4_c H4N

ADtr
ad_e <=> ad_c
{}
ad_e C2H5NO
ad_c C2H5NO



Now these reactions are balanced

In [64]:
get_metabolite_info(sorted_multiple_formula_metabolites[6])

alltn_e Allantoin C4H5N4O3;C4H6N4O3
Allantoin transport in via proton symport
alltn_e + h_e <=> alltn_c + h_c
{}
alltn_e C4H5N4O3;C4H6N4O3
h_e H
alltn_c C4H5N4O3;C4H6N4O3
h_c H

R_EX_alltn_e
alltn_e --> 
{'C': -8.0, 'H': -11.0, 'N': -8.0, 'O': -6.0}
alltn_e C4H5N4O3;C4H6N4O3



Can't do much with this

In [65]:
get_metabolite_info(sorted_multiple_formula_metabolites[7])

fdxox_c Oxidized ferredoxin X;Fe8S8X
2-oxoglutarate synthase (rev)
akg_c + coa_c + fdxox_c + h_c --> co2_c + fdxrd_c + succoa_c
{'H': 2.0, 'S': 2.0, 'X': 1.0, 'Fe': 2.0}
akg_c C5H4O5
coa_c C21H32N7O16P3S
h_c H
fdxox_c X;Fe8S8X
co2_c CO2
succoa_c C25H35N7O19P3S
fdxrd_c XH2;Fe8S8XH2;Fe2S2X

FRDO
fdxrd_c + nadp_c <=> fdxox_c + h_c + nadph_c
{'H': -2.0, 'X': -1.0, 'Fe': -2.0, 'S': -2.0}
nadp_c C21H25N7O17P3
fdxrd_c XH2;Fe8S8XH2;Fe2S2X
h_c H
nadph_c C21H26N7O17P3
fdxox_c X;Fe8S8X



Both reactions have multiple undefined formulas so no strong conclusions can be made

In [66]:
get_metabolite_info(sorted_multiple_formula_metabolites[8])

so3_p Sulfite O3S;HSO3
Sulfite transport via diffusion (extracellular to periplasm)
so3_e <=> so3_p
{}
so3_e O3S;HSO3
so3_p O3S;HSO3

Sulfite transport via ABC system (periplasm)
atp_c + h2o_c + so3_p --> adp_c + h_c + pi_c + so3_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
so3_p O3S;HSO3
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
so3_c O3S;HSO3



Since Sulfite appears to be only involved in transport it might not matter what its formula is

In [67]:
get_metabolite_info(sorted_multiple_formula_metabolites[9])

tsul_p Thiosulfate HS2O3;O3S2
Thiosulfate transport via diffusion (extracellular to periplasm)
tsul_e <=> tsul_p
{}
tsul_e HS2O3;O3S2
tsul_p HS2O3;O3S2

Thiosulfate transport via ABC system (periplasm)
atp_c + h2o_c + tsul_p --> adp_c + h_c + pi_c + tsul_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
tsul_p HS2O3;O3S2
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
tsul_c HS2O3;O3S2



Since Thiosulfite appears to be only involved in transport it might not matter what its formula is

In [68]:
get_metabolite_info(sorted_multiple_formula_metabolites[10])

fol_c Folate C19H17N7O6;C19H18N7O6
FOLt
fol_e + h_e <=> fol_c + h_c
{}
h_e H
fol_e C19H17N7O6;C19H18N7O6
h_c H
fol_c C19H17N7O6;C19H18N7O6

FOLR2
fol_c + nadph_c --> dhf_c + nadp_c
{'C': -19.0, 'H': -17.0, 'N': -7.0, 'O': -6.0}
nadph_c C21H26N7O17P3
fol_c C19H17N7O6;C19H18N7O6
dhf_c C19H19N7O6
nadp_c C21H25N7O17P3



In [74]:
model.metabolites.get_by_id('fol_c').formula = 'C19H18N7O6'
model.metabolites.get_by_id('fol_e').formula = 'C19H18N7O6'

In [75]:
get_metabolite_info(sorted_multiple_formula_metabolites[10])

fol_c Folate C19H18N7O6
FOLt
fol_e + h_e <=> fol_c + h_c
{}
h_e H
fol_e C19H18N7O6
h_c H
fol_c C19H18N7O6

FOLR2
fol_c + nadph_c --> dhf_c + nadp_c
{}
nadph_c C21H26N7O17P3
fol_c C19H18N7O6
dhf_c C19H19N7O6
nadp_c C21H25N7O17P3



Now both reactions are balanced

### fol_c one did not follow the pattern that the proper formula is the formula not equal to the mass error

In [77]:
get_metabolite_info(sorted_multiple_formula_metabolites[11])

gthrd_p Reduced glutathione C10H15N3O6S;C10H16N3O6S
Reduced glutathione via ABC system (periplasm)
atp_c + gthrd_p + h2o_c --> adp_c + gthrd_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
gthrd_p C10H15N3O6S;C10H16N3O6S
adp_c C10H12N5O10P2
gthrd_c C10H15N3O6S;C10H16N3O6S
h_c H
pi_c HPO4

Glutathione export via ABC system (cytoplasm to periplasm)
atp_c + gthrd_c + h2o_c --> adp_c + gthrd_p + h_c + pi_c
{}
atp_c C10H12N5O13P3
gthrd_c C10H15N3O6S;C10H16N3O6S
h2o_c H2O
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
gthrd_p C10H15N3O6S;C10H16N3O6S



Can't do much since both reactions have multiple undefined metabolites

In [78]:
get_metabolite_info(sorted_multiple_formula_metabolites[12])

pheme_c Protoheme C34H32FeN4O4;C34FeH30N4O4;C34H30FeN4O4
Ferrochelatase
fe2_c + ppp9_c --> 2.0 h_c + pheme_c
{'Fe': 2.0, 'C': 68.0, 'H': 62.0, 'N': 8.0, 'O': 8.0}
fe2_c Fe
ppp9_c C34H32N4O4
h_c H
pheme_c C34H32FeN4O4;C34FeH30N4O4;C34H30FeN4O4

Protoheme transport via ABC system (periplasm)
atp_c + h2o_c + pheme_c --> adp_c + h_c + pheme_p + pi_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
pheme_c C34H32FeN4O4;C34FeH30N4O4;C34H30FeN4O4
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
pheme_p C34H32FeN4O4;C34FeH30N4O4;C34H30FeN4O4



## This one is interesting since pheme_p has 3 options
Solved by trying formulas one at a time until mass error are all zero

In [82]:
model.metabolites.get_by_id('pheme_c').formula = 'C34FeH30N4O4'
model.metabolites.get_by_id('pheme_p').formula = 'C34FeH30N4O4'

In [84]:
get_metabolite_info(sorted_multiple_formula_metabolites[12])

pheme_c Protoheme C34FeH30N4O4
Ferrochelatase
fe2_c + ppp9_c --> 2.0 h_c + pheme_c
{}
fe2_c Fe
ppp9_c C34H32N4O4
h_c H
pheme_c C34FeH30N4O4

Protoheme transport via ABC system (periplasm)
atp_c + h2o_c + pheme_c --> adp_c + h_c + pheme_p + pi_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
pheme_c C34FeH30N4O4
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
pheme_p C34FeH30N4O4



In [85]:
get_metabolite_info(sorted_multiple_formula_metabolites[13])

fe3dhbzs_c Ferric 2,3-dihydroxybenzoylserine C10FeH11NO6;C10H10NO6Fe
Release of Fe(III) from ferric 2,3-dihydroxybenzoylserine
fe3dhbzs_c --> 23dhbzs_c + fe3_c
{'Fe': -1.0}
fe3dhbzs_c C10FeH11NO6;C10H10NO6Fe
23dhbzs_c C10H10NO6;C10H11NO6
fe3_c Fe

Ferric 2,3-dihydroxybenzoylserine transport via ABC system (periplasm)
atp_c + fe3dhbzs_p + h2o_c --> adp_c + fe3dhbzs_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
fe3dhbzs_p C10FeH11NO6;C10H10NO6Fe
adp_c C10H12N5O10P2
fe3dhbzs_c C10FeH11NO6;C10H10NO6Fe
h_c H
pi_c HPO4



Can't do much since both have multiple undefined formulas

In [86]:
get_metabolite_info(sorted_multiple_formula_metabolites[14])

fe3dhbzs_p Ferric 2,3-dihydroxybenzoylserine C10FeH11NO6;C10H10NO6Fe
Ferric 2,3-dihydroxybenzoylserine transport via ton system (extracellular)
fe3dhbzs_e + h_p --> fe3dhbzs_p + h_c
{}
fe3dhbzs_e C10FeH11NO6;C10H10NO6Fe
h_p H
h_c H
fe3dhbzs_p C10FeH11NO6;C10H10NO6Fe

Ferric 2,3-dihydroxybenzoylserine transport via ABC system (periplasm)
atp_c + fe3dhbzs_p + h2o_c --> adp_c + fe3dhbzs_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
fe3dhbzs_p C10FeH11NO6;C10H10NO6Fe
adp_c C10H12N5O10P2
fe3dhbzs_c C10FeH11NO6;C10H10NO6Fe
h_c H
pi_c HPO4



In [87]:
get_metabolite_info(sorted_multiple_formula_metabolites[15])

fe3dhbzs_e Ferric 2,3-dihydroxybenzoylserine C10FeH11NO6;C10H10NO6Fe
Ferric 2,3-dihydroxybenzoylserine transport via ton system (extracellular)
fe3dhbzs_e + h_p --> fe3dhbzs_p + h_c
{}
fe3dhbzs_e C10FeH11NO6;C10H10NO6Fe
h_p H
h_c H
fe3dhbzs_p C10FeH11NO6;C10H10NO6Fe

R_EX_fe3dhbzs_e
fe3dhbzs_e --> 
{'C': -20.0, 'Fe': -2.0, 'H': -21.0, 'N': -2.0, 'O': -12.0}
fe3dhbzs_e C10FeH11NO6;C10H10NO6Fe



In [88]:
get_metabolite_info(sorted_multiple_formula_metabolites[16])

forglu_c N-Formimidoyl-L-glutamate C6H8N2O4
Formimidoylglutamase
forglu_c + h2o_c + h_c --> frmd_c + glu__L_c
{'H': 4.0, 'O': 1.0, 'C': 1.0, 'N': 1.0}
h_c H
h2o_c H2O
forglu_c C6H8N2O4
glu__L_c C5H8NO4
frmd_c CH3NO;CH4NO

Imidazolonepropionase
4izp_c + h2o_c --> forglu_c + h_c
{}
h2o_c H2O
4izp_c C6H7N2O3
h_c H
forglu_c C6H8N2O4



In [89]:
model.metabolites.get_by_id('frmd_c').formula = 'CH3NO'

In [90]:
get_metabolite_info(sorted_multiple_formula_metabolites[16])

forglu_c N-Formimidoyl-L-glutamate C6H8N2O4
Formimidoylglutamase
forglu_c + h2o_c + h_c --> frmd_c + glu__L_c
{}
h_c H
h2o_c H2O
forglu_c C6H8N2O4
glu__L_c C5H8NO4
frmd_c CH3NO

Imidazolonepropionase
4izp_c + h2o_c --> forglu_c + h_c
{}
h2o_c H2O
4izp_c C6H7N2O3
h_c H
forglu_c C6H8N2O4



In [91]:
get_metabolite_info(sorted_multiple_formula_metabolites[17])

frmd_c Formamide CH3NO
Formimidoylglutamase
forglu_c + h2o_c + h_c --> frmd_c + glu__L_c
{}
h_c H
h2o_c H2O
forglu_c C6H8N2O4
glu__L_c C5H8NO4
frmd_c CH3NO

FORAMD
frmd_c + h2o_c --> for_c + nh4_c
{}
h2o_c H2O
frmd_c CH3NO
for_c CHO2
nh4_c H4N



In [92]:
get_metabolite_info(sorted_multiple_formula_metabolites[18])

fol_e Folate C19H18N7O6
R_EX_fol_e
fol_e --> 
{'C': -19.0, 'H': -18.0, 'N': -7.0, 'O': -6.0}
fol_e C19H18N7O6

FOLt
fol_e + h_e <=> fol_c + h_c
{}
h_e H
fol_e C19H18N7O6
h_c H
fol_c C19H18N7O6



In [94]:
get_metabolite_info(sorted_multiple_formula_metabolites[19])

glutrna_c L-Glutamyl-tRNA(Glu) C5H7NO3R;C5H8NO4X
Glutamyl-tRNA synthetase
atp_c + glu__L_c + trnaglu_c --> amp_c + glutrna_c + ppi_c
{'C': -10.0, 'H': -14.0, 'N': -4.0, 'O': -7.0, 'P': -1.0, 'R': -1.0}
atp_c C10H12N5O13P3
glu__L_c C5H8NO4
trnaglu_c R;HOX;C15H21N5O10PR
amp_c C10H12N5O7P
glutrna_c C5H7NO3R;C5H8NO4X
ppi_c HO7P2

Glutamyl-tRNA reductase
glutrna_c + h_c + nadph_c --> glu1sa_c + nadp_c + trnaglu_c
{'C': 10.0, 'H': 14.0, 'N': 4.0, 'O': 7.0, 'R': 1.0, 'P': 1.0}
glutrna_c C5H7NO3R;C5H8NO4X
h_c H
nadph_c C21H26N7O17P3
glu1sa_c C5H9NO3
nadp_c C21H25N7O17P3
trnaglu_c R;HOX;C15H21N5O10PR



In [95]:
get_metabolite_info(sorted_multiple_formula_metabolites[20])

trnaglu_c TRNA (Glu) R;HOX;C15H21N5O10PR
Glutamyl-tRNA synthetase
atp_c + glu__L_c + trnaglu_c --> amp_c + glutrna_c + ppi_c
{'C': -10.0, 'H': -14.0, 'N': -4.0, 'O': -7.0, 'P': -1.0, 'R': -1.0}
atp_c C10H12N5O13P3
glu__L_c C5H8NO4
trnaglu_c R;HOX;C15H21N5O10PR
amp_c C10H12N5O7P
glutrna_c C5H7NO3R;C5H8NO4X
ppi_c HO7P2

Glutamyl-tRNA reductase
glutrna_c + h_c + nadph_c --> glu1sa_c + nadp_c + trnaglu_c
{'C': 10.0, 'H': 14.0, 'N': 4.0, 'O': 7.0, 'R': 1.0, 'P': 1.0}
glutrna_c C5H7NO3R;C5H8NO4X
h_c H
nadph_c C21H26N7O17P3
glu1sa_c C5H9NO3
nadp_c C21H25N7O17P3
trnaglu_c R;HOX;C15H21N5O10PR



In [96]:
get_metabolite_info(sorted_multiple_formula_metabolites[21])

lgt__S_c (R)-S-Lactoylglutathione C13H19N3O8S;C13H20N3O8S
Hydroxyacylglutathione hydrolase
h2o_c + lgt__S_c --> gthrd_c + h_c + lac__D_c
{'H': -4.0, 'O': -2.0, 'C': -3.0}
h2o_c H2O
lgt__S_c C13H19N3O8S;C13H20N3O8S
gthrd_c C10H15N3O6S;C10H16N3O6S
h_c H
lac__D_c C3H5O3

D-Lactaldehyde NAD+ oxidoreductase (glutathione-formylating)
gthrd_c + lald__D_c + nad_c <=> h_c + lgt__S_c + nadh_c
{'C': 3.0, 'H': 4.0, 'O': 2.0}
gthrd_c C10H15N3O6S;C10H16N3O6S
lald__D_c C3H6O2
nad_c C21H26N7O14P2
h_c H
lgt__S_c C13H19N3O8S;C13H20N3O8S
nadh_c C21H27N7O14P2



In [97]:
get_metabolite_info(sorted_multiple_formula_metabolites[22])

gsn_e Guanosine C10H13N5O5;C10H12N5O5
Guanosine transport in via proton symport
gsn_e + h_e --> gsn_c + h_c
{'C': -10.0, 'H': -12.0, 'N': -5.0, 'O': -5.0}
gsn_e C10H13N5O5;C10H12N5O5
h_e H
gsn_c C10H13N5O5
h_c H

R_EX_gsn_e
gsn_e --> 
{'C': -20.0, 'H': -25.0, 'N': -10.0, 'O': -10.0}
gsn_e C10H13N5O5;C10H12N5O5



Could have defined this earlier

In [98]:
model.metabolites.get_by_id('gsn_e').formula = 'C10H13N5O5'

In [99]:
get_metabolite_info(sorted_multiple_formula_metabolites[22])

gsn_e Guanosine C10H13N5O5
Guanosine transport in via proton symport
gsn_e + h_e --> gsn_c + h_c
{}
gsn_e C10H13N5O5
h_e H
gsn_c C10H13N5O5
h_c H

R_EX_gsn_e
gsn_e --> 
{'C': -10.0, 'H': -13.0, 'N': -5.0, 'O': -5.0}
gsn_e C10H13N5O5



In [100]:
get_metabolite_info(sorted_multiple_formula_metabolites[23])

gua_e Guanine C5H4N5O;C5H5N5O
Guanine reversible transport via proton symport
gua_e + h_e --> gua_c + h_c
{}
gua_e C5H4N5O;C5H5N5O
h_e H
gua_c C5H4N5O;C5H5N5O
h_c H

R_EX_gua_e
gua_e --> 
{'C': -10.0, 'H': -9.0, 'N': -10.0, 'O': -2.0}
gua_e C5H4N5O;C5H5N5O



In [101]:
get_metabolite_info(sorted_multiple_formula_metabolites[24])

h2s_e Hydrogen sulfide HS;H2S
H2St
h2s_e <=> h2s_c
{}
h2s_e HS;H2S
h2s_c HS;H2S

R_EX_h2s_e
h2s_e --> 
{'H': -3.0, 'S': -2.0}
h2s_e HS;H2S



Might not matter what the formula is

In [103]:
get_metabolite_info(sorted_multiple_formula_metabolites[25])

pheme_e Protoheme C34H32FeN4O4;C34FeH30N4O4;C34H30FeN4O4
R_EX_pheme_e
pheme_e --> 
{'C': -102.0, 'H': -92.0, 'Fe': -3.0, 'N': -12.0, 'O': -12.0}
pheme_e C34H32FeN4O4;C34FeH30N4O4;C34H30FeN4O4

Protoheme transport irreversible out via diffusion (periplasm to extracellular)
pheme_p --> pheme_e
{'C': 68.0, 'Fe': 2.0, 'H': 62.0, 'N': 8.0, 'O': 8.0}
pheme_p C34FeH30N4O4
pheme_e C34H32FeN4O4;C34FeH30N4O4;C34H30FeN4O4



Could have defined this earlier

In [104]:
model.metabolites.get_by_id('pheme_e').formula = 'C34FeH30N4O4'

In [105]:
get_metabolite_info(sorted_multiple_formula_metabolites[25])

pheme_e Protoheme C34FeH30N4O4
R_EX_pheme_e
pheme_e --> 
{'C': -34.0, 'Fe': -1.0, 'H': -30.0, 'N': -4.0, 'O': -4.0}
pheme_e C34FeH30N4O4

Protoheme transport irreversible out via diffusion (periplasm to extracellular)
pheme_p --> pheme_e
{}
pheme_p C34FeH30N4O4
pheme_e C34FeH30N4O4



In [106]:
get_metabolite_info(sorted_multiple_formula_metabolites[26])

taur_e Taurine C2H7NO3S;C2H6NO3S
Taurine transport via ABC system
atp_c + h2o_c + taur_e --> adp_c + h_c + pi_c + taur_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
taur_e C2H7NO3S;C2H6NO3S
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
taur_c C2H7NO3S;C2H6NO3S

R_EX_taur_e
taur_e --> 
{'C': -4.0, 'H': -13.0, 'N': -2.0, 'O': -6.0, 'S': -2.0}
taur_e C2H7NO3S;C2H6NO3S



In [107]:
get_metabolite_info(sorted_multiple_formula_metabolites[27])

prlp_c 5-[(5-phospho-1-deoxyribulos-1-ylamino)methylideneamino]-1-(5-phosphoribosyl)imidazole-4-carboxamide C15H22N5O15P2;C15H21N5O15P2
1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase
prfp_c <=> prlp_c
{}
prfp_c C15H22N5O15P2;C15H21N5O15P2
prlp_c C15H22N5O15P2;C15H21N5O15P2

Imidazole-glycerol-3-phosphate synthase
gln__L_c + prlp_c --> aicar_c + eig3p_c + glu__L_c + h_c
{'C': -15.0, 'H': -22.0, 'N': -5.0, 'O': -15.0, 'P': -2.0}
gln__L_c C5H10N2O3
prlp_c C15H22N5O15P2;C15H21N5O15P2
aicar_c C9H13N4O8P
eig3p_c C6H9N2O6P
glu__L_c C5H8NO4
h_c H



In [108]:
model.metabolites.get_by_id('prlp_c').formula = 'C15H21N5O15P2'
model.metabolites.get_by_id('prfp_c').formula = 'C15H21N5O15P2'

In [109]:
get_metabolite_info(sorted_multiple_formula_metabolites[27])

prlp_c 5-[(5-phospho-1-deoxyribulos-1-ylamino)methylideneamino]-1-(5-phosphoribosyl)imidazole-4-carboxamide C15H21N5O15P2
1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase
prfp_c <=> prlp_c
{}
prfp_c C15H21N5O15P2
prlp_c C15H21N5O15P2

Imidazole-glycerol-3-phosphate synthase
gln__L_c + prlp_c --> aicar_c + eig3p_c + glu__L_c + h_c
{}
gln__L_c C5H10N2O3
prlp_c C15H21N5O15P2
aicar_c C9H13N4O8P
eig3p_c C6H9N2O6P
glu__L_c C5H8NO4
h_c H



In [110]:
get_metabolite_info(sorted_multiple_formula_metabolites[28])

rnam_c N Ribosylnicotinamide C11H15N2O5 C11H15N2O5;C11H14N2O5
Nicotinamide ribonucleotide phosphohydrolase
h2o_c + nmn_c --> pi_c + rnam_c
{'H': 1.0, 'O': -3.0, 'P': -1.0}
h2o_c H2O
nmn_c C11H14N2O8P;C11H13N2O8P
pi_c HPO4
rnam_c C11H15N2O5;C11H14N2O5

Ribosylnicotinamide kinase
atp_c + rnam_c --> adp_c + h_c + nmn_c
{'H': -1.0, 'O': 3.0, 'P': 1.0}
atp_c C10H12N5O13P3
rnam_c C11H15N2O5;C11H14N2O5
adp_c C10H12N5O10P2
h_c H
nmn_c C11H14N2O8P;C11H13N2O8P



In [111]:
get_metabolite_info(sorted_multiple_formula_metabolites[29])

udcpo4_c Undecaprenyl diphosphate galactose-rhamnose-mannose-abequose C79H132O24P2;C79H130O24P2
Abequosyl transferase LPS O antigen biosynthesis 
cdpabeq_c + udcdpgalrmnman_c --> cdp_c + h_c + udcpo4_c
{'C': 79.0, 'H': 132.0, 'O': 24.0, 'P': 2.0}
udcdpgalrmnman_c C73H120O21P2
cdpabeq_c C15H23N3O14P2
cdp_c C9H12N3O11P2
h_c H
udcpo4_c C79H132O24P2;C79H130O24P2

O-antigen Acetyl-Transferase
accoa_c + udcpo4_c --> coa_c + udcpo5_c
{'C': -79.0, 'H': -130.0, 'O': -24.0, 'P': -2.0}
accoa_c C23H34N7O17P3S
udcpo4_c C79H132O24P2;C79H130O24P2
coa_c C21H32N7O16P3S
udcpo5_c C81H134O25P2



In [112]:
model.metabolites.get_by_id('udcpo4_c').formula = 'C79H130O24P2'

In [113]:
get_metabolite_info(sorted_multiple_formula_metabolites[29])

udcpo4_c Undecaprenyl diphosphate galactose-rhamnose-mannose-abequose C79H130O24P2
Abequosyl transferase LPS O antigen biosynthesis 
cdpabeq_c + udcdpgalrmnman_c --> cdp_c + h_c + udcpo4_c
{}
udcdpgalrmnman_c C73H120O21P2
cdpabeq_c C15H23N3O14P2
cdp_c C9H12N3O11P2
h_c H
udcpo4_c C79H130O24P2

O-antigen Acetyl-Transferase
accoa_c + udcpo4_c --> coa_c + udcpo5_c
{'H': 2.0}
accoa_c C23H34N7O17P3S
udcpo4_c C79H130O24P2
coa_c C21H32N7O16P3S
udcpo5_c C81H134O25P2



### This one is a serious problem since there are conflicting definitions. To make fit, another formula will need to change
Reset it for now

In [114]:
model.metabolites.get_by_id('udcpo4_c').formula = 'C79H132O24P2;C79H130O24P2'

In [115]:
get_metabolite_info(sorted_multiple_formula_metabolites[30])

pheme_p Protoheme C34FeH30N4O4
Protoheme transport via ABC system (periplasm)
atp_c + h2o_c + pheme_c --> adp_c + h_c + pheme_p + pi_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
pheme_c C34FeH30N4O4
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
pheme_p C34FeH30N4O4

Protoheme transport irreversible out via diffusion (periplasm to extracellular)
pheme_p --> pheme_e
{}
pheme_p C34FeH30N4O4
pheme_e C34FeH30N4O4



In [116]:
get_metabolite_info(sorted_multiple_formula_metabolites[31])

prfp_c 1-(5-Phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino]imidazole-4-carboxamide C15H21N5O15P2
1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase
prfp_c <=> prlp_c
{}
prfp_c C15H21N5O15P2
prlp_c C15H21N5O15P2

Phosphoribosyl-AMP cyclohydrolase
h2o_c + prbamp_c --> prfp_c
{}
h2o_c H2O
prbamp_c C15H19N5O14P2
prfp_c C15H21N5O15P2



In [117]:
get_metabolite_info(sorted_multiple_formula_metabolites[32])

urea_e Urea CH4N2O;CH5N2O
Urea transport via facilitate diffusion
urea_e <=> urea_c
{}
urea_e CH4N2O;CH5N2O
urea_c CH4N2O;CH5N2O

R_EX_urea_e
urea_e --> 
{'C': -2.0, 'H': -9.0, 'N': -4.0, 'O': -2.0}
urea_e CH4N2O;CH5N2O



In [118]:
get_metabolite_info(sorted_multiple_formula_metabolites[33])

so3_e Sulfite O3S;HSO3
Sulfite transport via diffusion (extracellular to periplasm)
so3_e <=> so3_p
{}
so3_e O3S;HSO3
so3_p O3S;HSO3

R_EX_so3_e
so3_e --> 
{'O': -6.0, 'S': -2.0, 'H': -1.0}
so3_e O3S;HSO3



In [119]:
get_metabolite_info(sorted_multiple_formula_metabolites[34])

urea_p Urea CH4N2O;CH5N2O
Urea transport via facilitate diffusion (periplasm)
urea_p <=> urea_c
{}
urea_p CH4N2O;CH5N2O
urea_c CH4N2O;CH5N2O

Urea transport via ABC system (periplasm)
atp_c + h2o_c + urea_p --> adp_c + h_c + pi_c + urea_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
urea_p CH4N2O;CH5N2O
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
urea_c CH4N2O;CH5N2O



In [120]:
get_metabolite_info(sorted_multiple_formula_metabolites[35])

4cml_c 4-Carboxymuconolactone C7H4O6;C7H6O6
4CMLCL kt
4cml_c + h_c --> 5odhf2a_c + co2_c
{'H': -6.0, 'C': -7.0, 'O': -6.0}
h_c H
4cml_c C7H4O6;C7H6O6
co2_c CO2
5odhf2a_c C6H5O4

CMLDC
4cml_c --> 5odhf2a_c + co2_c + h_c
{'C': -7.0, 'H': -4.0, 'O': -6.0}
4cml_c C7H4O6;C7H6O6
co2_c CO2
h_c H
5odhf2a_c C6H5O4

MUCCY kt
CCbuttc_c + h_c --> 4cml_c
{'H': 6.0, 'C': 7.0, 'O': 6.0}
h_c H
CCbuttc_c C7H3O6
4cml_c C7H4O6;C7H6O6



In [121]:
get_metabolite_info(sorted_multiple_formula_metabolites[36])

dscl_c Dihydrosirohydrochlorin C42H40N4O16;C42H41N4O16
Uroporphyrinogen methyltransferase
2.0 amet_c + uppg3_c --> 2.0 ahcys_c + dscl_c + h_c
{'C': 42.0, 'H': 40.0, 'N': 4.0, 'O': 16.0}
amet_c C15H23N6O5S
uppg3_c C40H36N4O16
ahcys_c C14H20N6O5S
dscl_c C42H40N4O16;C42H41N4O16
h_c H

Sirohydrochlorin dehydrogenase (NAD)
dscl_c + nad_c --> h_c + nadh_c + scl_c
{'C': -42.0, 'H': -40.0, 'N': -4.0, 'O': -16.0}
dscl_c C42H40N4O16;C42H41N4O16
nad_c C21H26N7O14P2
h_c H
nadh_c C21H27N7O14P2
scl_c C42H39N4O16

SHCHD3
dscl_c + nad_c --> h_c + nadh_c + scl2_c
{'C': -42.0, 'H': -41.0, 'N': -4.0, 'O': -16.0}
dscl_c C42H40N4O16;C42H41N4O16
nad_c C21H26N7O14P2
h_c H
nadh_c C21H27N7O14P2
scl2_c C42H38N4O16



### Problem due to inconsistent formulas. Need to change another metabolites formula or the reaction

In [126]:
get_metabolite_info(sorted_multiple_formula_metabolites[37])

gar_c N1-(5-Phospho-D-ribosyl)glycinamide C7H14N2O8P;C7H13N2O8P
GAR transformylase-T
atp_c + for_c + gar_c --> adp_c + fgam_c + h_c + pi_c
{'C': 1.0, 'H': -1.0, 'O': 1.0}
atp_c C10H12N5O13P3
for_c CHO2
gar_c C7H14N2O8P;C7H13N2O8P
adp_c C10H12N5O10P2
fgam_c C8H12N2O9P;C8H13N2O9P
h_c H
pi_c HPO4

Phosphoribosylglycinamide synthase
atp_c + gly_c + pram_c <=> adp_c + gar_c + h_c + pi_c
{'C': 7.0, 'H': 13.0, 'N': 2.0, 'O': 8.0, 'P': 1.0}
atp_c C10H12N5O13P3
gly_c C2H5NO2
pram_c C5H11NO7P
adp_c C10H12N5O10P2
gar_c C7H14N2O8P;C7H13N2O8P
h_c H
pi_c HPO4

Phosphoribosylglycinamide formyltransferase
10fthf_c + gar_c <=> fgam_c + h_c + thf_c
{'C': 1.0, 'H': -1.0, 'O': 1.0}
10fthf_c C20H21N7O7
gar_c C7H14N2O8P;C7H13N2O8P
fgam_c C8H12N2O9P;C8H13N2O9P
h_c H
thf_c C19H21N7O6



In [127]:
model.metabolites.get_by_id('gar_c').formula = 'C7H14N2O8P'

In [128]:
get_metabolite_info(sorted_multiple_formula_metabolites[37])

gar_c N1-(5-Phospho-D-ribosyl)glycinamide C7H14N2O8P
GAR transformylase-T
atp_c + for_c + gar_c --> adp_c + fgam_c + h_c + pi_c
{'C': 8.0, 'H': 12.0, 'N': 2.0, 'O': 9.0, 'P': 1.0}
atp_c C10H12N5O13P3
for_c CHO2
gar_c C7H14N2O8P
adp_c C10H12N5O10P2
fgam_c C8H12N2O9P;C8H13N2O9P
h_c H
pi_c HPO4

Phosphoribosylglycinamide synthase
atp_c + gly_c + pram_c <=> adp_c + gar_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
gly_c C2H5NO2
pram_c C5H11NO7P
adp_c C10H12N5O10P2
gar_c C7H14N2O8P
h_c H
pi_c HPO4

Phosphoribosylglycinamide formyltransferase
10fthf_c + gar_c <=> fgam_c + h_c + thf_c
{'C': 8.0, 'H': 12.0, 'N': 2.0, 'O': 9.0, 'P': 1.0}
10fthf_c C20H21N7O7
gar_c C7H14N2O8P
fgam_c C8H12N2O9P;C8H13N2O9P
h_c H
thf_c C19H21N7O6



In [129]:
model.metabolites.get_by_id('fgam_c').formula = 'C8H13N2O9P'

In [130]:
get_metabolite_info(sorted_multiple_formula_metabolites[37])

gar_c N1-(5-Phospho-D-ribosyl)glycinamide C7H14N2O8P
GAR transformylase-T
atp_c + for_c + gar_c --> adp_c + fgam_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
for_c CHO2
gar_c C7H14N2O8P
adp_c C10H12N5O10P2
fgam_c C8H13N2O9P
h_c H
pi_c HPO4

Phosphoribosylglycinamide synthase
atp_c + gly_c + pram_c <=> adp_c + gar_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
gly_c C2H5NO2
pram_c C5H11NO7P
adp_c C10H12N5O10P2
gar_c C7H14N2O8P
h_c H
pi_c HPO4

Phosphoribosylglycinamide formyltransferase
10fthf_c + gar_c <=> fgam_c + h_c + thf_c
{}
10fthf_c C20H21N7O7
gar_c C7H14N2O8P
fgam_c C8H13N2O9P
h_c H
thf_c C19H21N7O6



In [131]:
get_metabolite_info(sorted_multiple_formula_metabolites[38])

pnto__R_e (R)-Pantothenate C9H16NO5;C9H15NO5
Pantothenate reversible transport via proton symport
h_e + pnto__R_e <=> h_c + pnto__R_c
{}
h_e H
pnto__R_e C9H16NO5;C9H15NO5
h_c H
pnto__R_c C9H16NO5;C9H15NO5

R_EX_pnto__R_e
pnto__R_e --> 
{'C': -18.0, 'H': -31.0, 'N': -2.0, 'O': -10.0}
pnto__R_e C9H16NO5;C9H15NO5

Pantothenate sodium symporter
na1_e + pnto__R_e <=> na1_c + pnto__R_c
{}
na1_e Na
pnto__R_e C9H16NO5;C9H15NO5
na1_c Na
pnto__R_c C9H16NO5;C9H15NO5



In [132]:
get_metabolite_info(sorted_multiple_formula_metabolites[39])

fpram_c 2-(Formamido)-N1-(5-phospho-D-ribosyl)acetamidine C8H14N3O8P;C8H15N3O8P
Phosphoribosylformylglycinamidine synthase
atp_c + fgam_c + gln__L_c + h2o_c --> adp_c + fpram_c + glu__L_c + h_c + pi_c
{'C': 8.0, 'H': 14.0, 'N': 3.0, 'O': 8.0, 'P': 1.0}
atp_c C10H12N5O13P3
fgam_c C8H13N2O9P
gln__L_c C5H10N2O3
h2o_c H2O
adp_c C10H12N5O10P2
fpram_c C8H14N3O8P;C8H15N3O8P
glu__L_c C5H8NO4
h_c H
pi_c HPO4

Phosphoribosylaminoimidazole synthase
atp_c + fpram_c --> adp_c + air_c + 2.0 h_c + pi_c
{'H': -1.0, 'O': -1.0}
atp_c C10H12N5O13P3
fpram_c C8H14N3O8P;C8H15N3O8P
adp_c C10H12N5O10P2
air_c C8H13N3O7P;C8H12N3O7P
h_c H
pi_c HPO4

Phosphoribosylformylglycinamidine synthase
atp_c + fgam_c + gln__L_c + h2o_c --> adp_c + fpram_c + glu__L_c + 2.0 h_c + pi_c
{'C': 8.0, 'H': 15.0, 'N': 3.0, 'O': 8.0, 'P': 1.0}
atp_c C10H12N5O13P3
fgam_c C8H13N2O9P
gln__L_c C5H10N2O3
h2o_c H2O
adp_c C10H12N5O10P2
fpram_c C8H14N3O8P;C8H15N3O8P
glu__L_c C5H8NO4
h_c H
pi_c HPO4



# There is an inconsistancy in this metabolites

In [133]:
get_metabolite_info(sorted_multiple_formula_metabolites[40])

tsul_e Thiosulfate HS2O3;O3S2
Thiosulfate transport via diffusion (extracellular to periplasm)
tsul_e <=> tsul_p
{}
tsul_e HS2O3;O3S2
tsul_p HS2O3;O3S2

R_EX_tsul_e
tsul_e --> 
{'H': -1.0, 'S': -4.0, 'O': -6.0}
tsul_e HS2O3;O3S2

TSULabc
atp_c + h2o_c + tsul_e --> adp_c + h_c + pi_c + tsul_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
tsul_e HS2O3;O3S2
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
tsul_c HS2O3;O3S2



In [134]:
get_metabolite_info(sorted_multiple_formula_metabolites[41])

34dhpha_c 3-4-Dihydroxyphenylacetate C8H7O4;C8H8O4
3 4 dihydroxyphenylacetate 2 3 dioxygenase
34dhpha_c + o2_c --> 5cmhmsa_c + h_c
{'O': -4.0, 'C': -8.0, 'H': -8.0}
o2_c O2
34dhpha_c C8H7O4;C8H8O4
h_c H
5cmhmsa_c C8H6O6

HPA monooxygenase
3hoxpac_c + h_c + nadh_c + o2_c --> 34dhpha_c + h2o_c + nad_c
{'H': 7.0, 'C': 8.0, 'O': 4.0}
h_c H
nadh_c C21H27N7O14P2
o2_c O2
3hoxpac_c C8H8O3
h2o_c H2O
nad_c C21H26N7O14P2
34dhpha_c C8H7O4;C8H8O4

4 hydroxyphenylacetate 3 monooxygenase nadh
4hphac_c + h_c + nadh_c + o2_c --> 34dhpha_c + h2o_c + nad_c
{'H': 8.0, 'C': 8.0, 'O': 4.0}
h_c H
nadh_c C21H27N7O14P2
o2_c O2
4hphac_c C8H7O3
h2o_c H2O
nad_c C21H26N7O14P2
34dhpha_c C8H7O4;C8H8O4

4 hydroxyphenylacetate 3 monooxygenase FADH2
4hphac_c + fadh2_c + o2_c --> 34dhpha_c + fad_c + h2o_c
{'C': 8.0, 'H': 8.0, 'O': 4.0}
fadh2_c C27H33N9O15P2
o2_c O2
4hphac_c C8H7O3
fad_c C27H31N9O15P2
h2o_c H2O
34dhpha_c C8H7O4;C8H8O4



## There is an inconsistancy

In [135]:
get_metabolite_info(sorted_multiple_formula_metabolites[42])

air_c 5-amino-1-(5-phospho-D-ribosyl)imidazole C8H13N3O7P;C8H12N3O7P
Phosphoribosylaminoimidazole synthase
atp_c + fpram_c --> adp_c + air_c + 2.0 h_c + pi_c
{'H': -1.0, 'O': -1.0}
atp_c C10H12N5O13P3
fpram_c C8H14N3O8P;C8H15N3O8P
adp_c C10H12N5O10P2
air_c C8H13N3O7P;C8H12N3O7P
h_c H
pi_c HPO4

Phosphoribosylaminoimidazole carboxylase
air_c + co2_c <=> 5aizc_c + h_c
{'C': -8.0, 'H': -13.0, 'N': -3.0, 'O': -7.0, 'P': -1.0}
air_c C8H13N3O7P;C8H12N3O7P
co2_c CO2
5aizc_c C9H11N3O9P
h_c H

4 amino 5 hydroxymethyl 2 methylpyrimidine synthetase
air_c + 2.0 h_c --> 4ahmmp_c + gcald_c + pi_c
{'C': -8.0, 'H': -13.0, 'N': -3.0, 'O': -7.0, 'P': -1.0}
air_c C8H13N3O7P;C8H12N3O7P
h_c H
4ahmmp_c C6H9N3O
gcald_c C2H4O2
pi_c HPO4

Phosphoribosylaminoimidazole carboxylase
air_c + atp_c + hco3_c --> 5caiz_c + adp_c + h_c + pi_c
{'C': -8.0, 'H': -13.0, 'N': -3.0, 'O': -7.0, 'P': -1.0}
air_c C8H13N3O7P;C8H12N3O7P
atp_c C10H12N5O13P3
hco3_c CHO3
5caiz_c C9H11N3O9P
adp_c C10H12N5O10P2
h_c H
pi_c HPO4



In [136]:
model.metabolites.get_by_id('air_c').formula = 'C8H12N3O7P'

In [137]:
get_metabolite_info(sorted_multiple_formula_metabolites[42])

air_c 5-amino-1-(5-phospho-D-ribosyl)imidazole C8H12N3O7P
Phosphoribosylaminoimidazole synthase
atp_c + fpram_c --> adp_c + air_c + 2.0 h_c + pi_c
{'C': -8.0, 'H': -14.0, 'N': -3.0, 'O': -8.0, 'P': -1.0}
atp_c C10H12N5O13P3
fpram_c C8H14N3O8P;C8H15N3O8P
adp_c C10H12N5O10P2
air_c C8H12N3O7P
h_c H
pi_c HPO4

Phosphoribosylaminoimidazole carboxylase
air_c + co2_c <=> 5aizc_c + h_c
{}
air_c C8H12N3O7P
co2_c CO2
5aizc_c C9H11N3O9P
h_c H

4 amino 5 hydroxymethyl 2 methylpyrimidine synthetase
air_c + 2.0 h_c --> 4ahmmp_c + gcald_c + pi_c
{}
air_c C8H12N3O7P
h_c H
4ahmmp_c C6H9N3O
gcald_c C2H4O2
pi_c HPO4

Phosphoribosylaminoimidazole carboxylase
air_c + atp_c + hco3_c --> 5caiz_c + adp_c + h_c + pi_c
{}
air_c C8H12N3O7P
atp_c C10H12N5O13P3
hco3_c CHO3
5caiz_c C9H11N3O9P
adp_c C10H12N5O10P2
h_c H
pi_c HPO4



Leave the fpram_c for another pass

In [138]:
get_metabolite_info(sorted_multiple_formula_metabolites[43])

5mthf_c 5-Methyltetrahydrofolate C20H23N7O6;C20H24N7O6
5,10-methylenetetrahydrofolate reductase (NADH)
h_c + mlthf_c + nadh_c --> 5mthf_c + nad_c
{'H': 24.0, 'C': 20.0, 'N': 7.0, 'O': 6.0}
h_c H
mlthf_c C20H21N7O6
nadh_c C21H27N7O14P2
5mthf_c C20H23N7O6;C20H24N7O6
nad_c C21H26N7O14P2

5 10 methylenetetrahydrofolatereductase  NADPH 
2.0 h_c + mlthf_c + nadph_c --> 5mthf_c + nadp_c
{'H': 23.0, 'C': 20.0, 'N': 7.0, 'O': 6.0}
h_c H
mlthf_c C20H21N7O6
nadph_c C21H26N7O17P3
5mthf_c C20H23N7O6;C20H24N7O6
nadp_c C21H25N7O17P3

Methionine synthase
5mthf_c + hcys__L_c --> h_c + met__L_c + thf_c
{'C': -20.0, 'H': -23.0, 'N': -7.0, 'O': -6.0}
5mthf_c C20H23N7O6;C20H24N7O6
hcys__L_c C4H9NO2S
h_c H
met__L_c C5H11NO2S
thf_c C19H21N7O6

5,10-methylenetetrahydrofolate reductase (NADH)
2.0 h_c + mlthf_c + nadh_c --> 5mthf_c + nad_c
{'H': 23.0, 'C': 20.0, 'N': 7.0, 'O': 6.0}
h_c H
mlthf_c C20H21N7O6
nadh_c C21H27N7O14P2
5mthf_c C20H23N7O6;C20H24N7O6
nad_c C21H26N7O14P2



### There is an inconsistancy

In [139]:
get_metabolite_info(sorted_multiple_formula_metabolites[44])

tsul_c Thiosulfate HS2O3;O3S2
O-acetyl-L-serine sulfhydrylase
acser_c + tsul_c --> ac_c + scys__L_c
{'H': -1.0, 'O': -3.0, 'S': -2.0}
acser_c C5H9NO4
tsul_c HS2O3;O3S2
ac_c C2H3O2
scys__L_c C3H6NO5S2

TSULabc
atp_c + h2o_c + tsul_e --> adp_c + h_c + pi_c + tsul_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
tsul_e HS2O3;O3S2
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
tsul_c HS2O3;O3S2

Thiosulfate transport via ABC system (periplasm)
atp_c + h2o_c + tsul_p --> adp_c + h_c + pi_c + tsul_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
tsul_p HS2O3;O3S2
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
tsul_c HS2O3;O3S2

Cysteine synthase  Thiosulfate 
acser_c + trdrd_c + tsul_c <=> ac_c + cys__L_c + h_c + so3_c + trdox_c
{'H': -2.0, 'S': -1.0}
acser_c C5H9NO4
trdrd_c XH2;C6H9NO2S2R2
tsul_c HS2O3;O3S2
ac_c C2H3O2
cys__L_c C3H7NO2S
h_c H
so3_c O3S;HSO3
trdox_c X;C6H7NO2S2R2



In [140]:
get_metabolite_info(sorted_multiple_formula_metabolites[45])

alltn_c Allantoin C4H5N4O3;C4H6N4O3
Allantoinase
alltn_c + h2o_c --> alltt_c + h_c
{'C': -4.0, 'H': -5.0, 'N': -4.0, 'O': -3.0}
alltn_c C4H5N4O3;C4H6N4O3
h2o_c H2O
alltt_c C4H7N4O4
h_c H

Allantoin transport in via proton symport
alltn_e + h_e <=> alltn_c + h_c
{}
alltn_e C4H5N4O3;C4H6N4O3
h_e H
alltn_c C4H5N4O3;C4H6N4O3
h_c H

Uricase
2.0 h2o_c + o2_c + urate_c --> alltn_c + co2_c + h2o2_c
{'H': 5.0, 'O': 3.0, 'C': 4.0, 'N': 4.0}
h2o_c H2O
o2_c O2
urate_c C5H4N4O3
alltn_c C4H5N4O3;C4H6N4O3
co2_c CO2
h2o2_c H2O2

Allantoin transport in via proton symport (periplasm)
alltn_p + h_p <=> alltn_c + h_c
{}
alltn_p C4H5N4O3;C4H6N4O3
h_p H
alltn_c C4H5N4O3;C4H6N4O3
h_c H



In [143]:
model.metabolites.get_by_id('alltn_c').formula = 'C4H6N4O3'
model.metabolites.get_by_id('alltn_p').formula = 'C4H6N4O3'
model.metabolites.get_by_id('alltn_e').formula = 'C4H6N4O3'

In [144]:
get_metabolite_info(sorted_multiple_formula_metabolites[45])

alltn_c Allantoin C4H6N4O3
Allantoinase
alltn_c + h2o_c --> alltt_c + h_c
{}
alltn_c C4H6N4O3
h2o_c H2O
alltt_c C4H7N4O4
h_c H

Allantoin transport in via proton symport
alltn_e + h_e <=> alltn_c + h_c
{}
alltn_e C4H6N4O3
h_e H
alltn_c C4H6N4O3
h_c H

Uricase
2.0 h2o_c + o2_c + urate_c --> alltn_c + co2_c + h2o2_c
{}
h2o_c H2O
o2_c O2
urate_c C5H4N4O3
alltn_c C4H6N4O3
co2_c CO2
h2o2_c H2O2

Allantoin transport in via proton symport (periplasm)
alltn_p + h_p <=> alltn_c + h_c
{}
alltn_p C4H6N4O3
h_p H
alltn_c C4H6N4O3
h_c H



In [145]:
get_metabolite_info(sorted_multiple_formula_metabolites[46])

nmn_c NMN C11H14N2O8P;C11H13N2O8P
NMN nucleosidase
h2o_c + nmn_c --> h_c + ncam_c + r5p_c
{'H': -13.0, 'O': -8.0, 'C': -11.0, 'N': -2.0, 'P': -1.0}
h2o_c H2O
nmn_c C11H14N2O8P;C11H13N2O8P
h_c H
ncam_c C6H6N2O
r5p_c C5H9O8P

Nicotinamide ribonucleotide phosphohydrolase
h2o_c + nmn_c --> pi_c + rnam_c
{'H': 1.0, 'O': -3.0, 'P': -1.0}
h2o_c H2O
nmn_c C11H14N2O8P;C11H13N2O8P
pi_c HPO4
rnam_c C11H15N2O5;C11H14N2O5

NAD diphosphatase
h2o_c + nad_c --> amp_c + 2.0 h_c + nmn_c
{'H': 13.0, 'O': 8.0, 'C': 11.0, 'N': 2.0, 'P': 1.0}
h2o_c H2O
nad_c C21H26N7O14P2
amp_c C10H12N5O7P
h_c H
nmn_c C11H14N2O8P;C11H13N2O8P

Ribosylnicotinamide kinase
atp_c + rnam_c --> adp_c + h_c + nmn_c
{'H': -1.0, 'O': 3.0, 'P': 1.0}
atp_c C10H12N5O13P3
rnam_c C11H15N2O5;C11H14N2O5
adp_c C10H12N5O10P2
h_c H
nmn_c C11H14N2O8P;C11H13N2O8P



In [146]:
model.metabolites.get_by_id('nmn_c').formula = 'C11H14N2O8P'

In [147]:
get_metabolite_info(sorted_multiple_formula_metabolites[46])

nmn_c NMN C11H14N2O8P
NMN nucleosidase
h2o_c + nmn_c --> h_c + ncam_c + r5p_c
{}
h2o_c H2O
nmn_c C11H14N2O8P
h_c H
ncam_c C6H6N2O
r5p_c C5H9O8P

Nicotinamide ribonucleotide phosphohydrolase
h2o_c + nmn_c --> pi_c + rnam_c
{'H': 14.0, 'O': 5.0, 'C': 11.0, 'N': 2.0}
h2o_c H2O
nmn_c C11H14N2O8P
pi_c HPO4
rnam_c C11H15N2O5;C11H14N2O5

NAD diphosphatase
h2o_c + nad_c --> amp_c + 2.0 h_c + nmn_c
{}
h2o_c H2O
nad_c C21H26N7O14P2
amp_c C10H12N5O7P
h_c H
nmn_c C11H14N2O8P

Ribosylnicotinamide kinase
atp_c + rnam_c --> adp_c + h_c + nmn_c
{'C': -11.0, 'H': -14.0, 'N': -2.0, 'O': -5.0}
atp_c C10H12N5O13P3
rnam_c C11H15N2O5;C11H14N2O5
adp_c C10H12N5O10P2
h_c H
nmn_c C11H14N2O8P



In [148]:
get_metabolite_info(sorted_multiple_formula_metabolites[47])

4ppcys_c N-((R)-4-Phosphopantothenoyl)-L-cysteine C12H20N2O9PS
Phosphopantothenoylcysteine decarboxylase
4ppcys_c + h_c --> co2_c + pan4p_c
{}
4ppcys_c C12H20N2O9PS
h_c H
co2_c CO2
pan4p_c C11H21N2O7PS

PPNCL3
4ppan_c + atp_c + cys__L_c --> 4ppcys_c + amp_c + h_c + ppi_c
{'C': -9.0, 'H': -14.0, 'N': -1.0, 'O': -8.0, 'P': -1.0}
4ppan_c C9H15NO8P;C9H14NO8P
atp_c C10H12N5O13P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
amp_c C10H12N5O7P
h_c H
ppi_c HO7P2

PPNCL
4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cdp_c + h_c + pi_c
{'C': -9.0, 'H': -14.0, 'N': -1.0, 'O': -8.0, 'P': -1.0}
4ppan_c C9H15NO8P;C9H14NO8P
ctp_c C9H12N3O14P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
cdp_c C9H12N3O11P2
h_c H
pi_c HPO4

Phosphopantothenate-cysteine ligase
4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cmp_c + h_c + ppi_c
{'C': -9.0, 'H': -14.0, 'N': -1.0, 'O': -8.0, 'P': -1.0}
4ppan_c C9H15NO8P;C9H14NO8P
ctp_c C9H12N3O14P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
cmp_c C9H12N3O8P
h_c H
ppi_c HO7P2



In [149]:
get_metabolite_info(sorted_multiple_formula_metabolites[48])

23dhba_c (2,3-Dihydroxybenzoyl)adenylate C17H17N5O10P;C17H16N5O10P
Enterochelin synthase
3.0 23dhba_c + 3.0 seramp_c --> 6.0 amp_c + enter_c + 6.0 h_c
{'C': -51.0, 'H': -51.0, 'N': -15.0, 'O': -30.0, 'P': -3.0}
23dhba_c C17H17N5O10P;C17H16N5O10P
seramp_c C13H19N6O9P
amp_c C10H12N5O7P
enter_c C30H27N3O15
h_c H

Enterochelin synthase
3.0 23dhba_c + 3.0 seramp_c --> 6.0 amp_c + enter_c + 9.0 h_c
{'C': -51.0, 'H': -48.0, 'N': -15.0, 'O': -30.0, 'P': -3.0}
23dhba_c C17H17N5O10P;C17H16N5O10P
seramp_c C13H19N6O9P
amp_c C10H12N5O7P
enter_c C30H27N3O15
h_c H

2,3-dihydroxybenzoate adenylate synthase
23dhb_c + atp_c --> 23dhba_c + ppi_c
{'C': 17.0, 'H': 17.0, 'O': 10.0, 'N': 5.0, 'P': 1.0}
23dhb_c C7H5O4
atp_c C10H12N5O13P3
23dhba_c C17H17N5O10P;C17H16N5O10P
ppi_c HO7P2

2,3-dihydroxybenzoate adenylate synthase
23dhb_c + atp_c + h_c --> 23dhba_c + ppi_c
{'C': 17.0, 'H': 16.0, 'O': 10.0, 'N': 5.0, 'P': 1.0}
23dhb_c C7H5O4
atp_c C10H12N5O13P3
h_c H
23dhba_c C17H17N5O10P;C17H16N5O10P
ppi_c HO7P2



There is a conflict

In [150]:
get_metabolite_info(sorted_multiple_formula_metabolites[49])

fgam_c N2-Formyl-N1-(5-phospho-D-ribosyl)glycinamide C8H13N2O9P
Phosphoribosylformylglycinamidine synthase
atp_c + fgam_c + gln__L_c + h2o_c --> adp_c + fpram_c + glu__L_c + h_c + pi_c
{'C': 8.0, 'H': 14.0, 'N': 3.0, 'O': 8.0, 'P': 1.0}
atp_c C10H12N5O13P3
fgam_c C8H13N2O9P
gln__L_c C5H10N2O3
h2o_c H2O
adp_c C10H12N5O10P2
fpram_c C8H14N3O8P;C8H15N3O8P
glu__L_c C5H8NO4
h_c H
pi_c HPO4

GAR transformylase-T
atp_c + for_c + gar_c --> adp_c + fgam_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
for_c CHO2
gar_c C7H14N2O8P
adp_c C10H12N5O10P2
fgam_c C8H13N2O9P
h_c H
pi_c HPO4

Phosphoribosylglycinamide formyltransferase
10fthf_c + gar_c <=> fgam_c + h_c + thf_c
{}
10fthf_c C20H21N7O7
gar_c C7H14N2O8P
fgam_c C8H13N2O9P
h_c H
thf_c C19H21N7O6

Phosphoribosylformylglycinamidine synthase
atp_c + fgam_c + gln__L_c + h2o_c --> adp_c + fpram_c + glu__L_c + 2.0 h_c + pi_c
{'C': 8.0, 'H': 15.0, 'N': 3.0, 'O': 8.0, 'P': 1.0}
atp_c C10H12N5O13P3
fgam_c C8H13N2O9P
gln__L_c C5H10N2O3
h2o_c H2O
adp_c C10H12N5O10P2

In [151]:
get_metabolite_info(sorted_multiple_formula_metabolites[50])

pnto__R_c (R)-Pantothenate C9H16NO5;C9H15NO5
Pantothenate sodium symporter
na1_e + pnto__R_e <=> na1_c + pnto__R_c
{}
na1_e Na
pnto__R_e C9H16NO5;C9H15NO5
na1_c Na
pnto__R_c C9H16NO5;C9H15NO5

Pantothenate synthase
ala_B_c + atp_c + pant__R_c --> amp_c + h_c + pnto__R_c + ppi_c
{'C': 9.0, 'H': 15.0, 'N': 1.0, 'O': 5.0}
ala_B_c C3H7NO2
atp_c C10H12N5O13P3
pant__R_c C6H11O4
amp_c C10H12N5O7P
h_c H
pnto__R_c C9H16NO5;C9H15NO5
ppi_c HO7P2

Pantothenate kinase
atp_c + pnto__R_c --> 4ppan_c + adp_c + h_c
{'H': -1.0, 'O': 3.0, 'P': 1.0}
atp_c C10H12N5O13P3
pnto__R_c C9H16NO5;C9H15NO5
4ppan_c C9H15NO8P;C9H14NO8P
adp_c C10H12N5O10P2
h_c H

Pantothenate reversible transport via proton symport
h_e + pnto__R_e <=> h_c + pnto__R_c
{}
h_e H
pnto__R_e C9H16NO5;C9H15NO5
h_c H
pnto__R_c C9H16NO5;C9H15NO5



In [152]:
model.metabolites.get_by_id('pnto__R_c').formula = 'C9H16NO5'
model.metabolites.get_by_id('pnto__R_e').formula = 'C9H16NO5'

In [153]:
get_metabolite_info(sorted_multiple_formula_metabolites[50])

pnto__R_c (R)-Pantothenate C9H16NO5
Pantothenate sodium symporter
na1_e + pnto__R_e <=> na1_c + pnto__R_c
{}
na1_e Na
pnto__R_e C9H16NO5
na1_c Na
pnto__R_c C9H16NO5

Pantothenate synthase
ala_B_c + atp_c + pant__R_c --> amp_c + h_c + pnto__R_c + ppi_c
{}
ala_B_c C3H7NO2
atp_c C10H12N5O13P3
pant__R_c C6H11O4
amp_c C10H12N5O7P
h_c H
pnto__R_c C9H16NO5
ppi_c HO7P2

Pantothenate kinase
atp_c + pnto__R_c --> 4ppan_c + adp_c + h_c
{'C': 9.0, 'H': 14.0, 'N': 1.0, 'O': 8.0, 'P': 1.0}
atp_c C10H12N5O13P3
pnto__R_c C9H16NO5
4ppan_c C9H15NO8P;C9H14NO8P
adp_c C10H12N5O10P2
h_c H

Pantothenate reversible transport via proton symport
h_e + pnto__R_e <=> h_c + pnto__R_c
{}
h_e H
pnto__R_e C9H16NO5
h_c H
pnto__R_c C9H16NO5



In [154]:
get_metabolite_info(sorted_multiple_formula_metabolites[51])

4ppan_c D-4'-Phosphopantothenate C9H15NO8P;C9H14NO8P
Phosphopantothenate-cysteine ligase
4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cmp_c + h_c + ppi_c
{'C': -9.0, 'H': -14.0, 'N': -1.0, 'O': -8.0, 'P': -1.0}
4ppan_c C9H15NO8P;C9H14NO8P
ctp_c C9H12N3O14P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
cmp_c C9H12N3O8P
h_c H
ppi_c HO7P2

PPNCL3
4ppan_c + atp_c + cys__L_c --> 4ppcys_c + amp_c + h_c + ppi_c
{'C': -9.0, 'H': -14.0, 'N': -1.0, 'O': -8.0, 'P': -1.0}
4ppan_c C9H15NO8P;C9H14NO8P
atp_c C10H12N5O13P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
amp_c C10H12N5O7P
h_c H
ppi_c HO7P2

Pantothenate kinase
atp_c + pnto__R_c --> 4ppan_c + adp_c + h_c
{'C': 9.0, 'H': 14.0, 'N': 1.0, 'O': 8.0, 'P': 1.0}
atp_c C10H12N5O13P3
pnto__R_c C9H16NO5
4ppan_c C9H15NO8P;C9H14NO8P
adp_c C10H12N5O10P2
h_c H

PPNCL
4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cdp_c + h_c + pi_c
{'C': -9.0, 'H': -14.0, 'N': -1.0, 'O': -8.0, 'P': -1.0}
4ppan_c C9H15NO8P;C9H14NO8P
ctp_c C9H12N3O14P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9

In [155]:
model.metabolites.get_by_id('4ppan_c').formula = 'C9H15NO8P'

In [156]:
get_metabolite_info(sorted_multiple_formula_metabolites[51])

4ppan_c D-4'-Phosphopantothenate C9H15NO8P
Phosphopantothenate-cysteine ligase
4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cmp_c + h_c + ppi_c
{}
4ppan_c C9H15NO8P
ctp_c C9H12N3O14P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
cmp_c C9H12N3O8P
h_c H
ppi_c HO7P2

PPNCL3
4ppan_c + atp_c + cys__L_c --> 4ppcys_c + amp_c + h_c + ppi_c
{}
4ppan_c C9H15NO8P
atp_c C10H12N5O13P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
amp_c C10H12N5O7P
h_c H
ppi_c HO7P2

Pantothenate kinase
atp_c + pnto__R_c --> 4ppan_c + adp_c + h_c
{}
atp_c C10H12N5O13P3
pnto__R_c C9H16NO5
4ppan_c C9H15NO8P
adp_c C10H12N5O10P2
h_c H

PPNCL
4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cdp_c + h_c + pi_c
{}
4ppan_c C9H15NO8P
ctp_c C9H12N3O14P3
cys__L_c C3H7NO2S
4ppcys_c C12H20N2O9PS
cdp_c C9H12N3O11P2
h_c H
pi_c HPO4



In [157]:
get_metabolite_info(sorted_multiple_formula_metabolites[52])

gsn_c Guanosine C10H13N5O5
Guanosine transport in via proton symport (periplasm)
gsn_p + h_p --> gsn_c + h_c
{}
gsn_p C10H13N5O5
h_p H
gsn_c C10H13N5O5
h_c H

Guanosine transport in via proton symport
gsn_e + h_e --> gsn_c + h_c
{}
gsn_e C10H13N5O5
h_e H
gsn_c C10H13N5O5
h_c H

Gnnuc
gsn_c + h2o_c --> gua_c + rib__D_c
{'C': 5.0, 'H': 4.0, 'N': 5.0, 'O': 1.0}
gsn_c C10H13N5O5
h2o_c H2O
gua_c C5H4N5O;C5H5N5O
rib__D_c C5H10O5

Nucleoside triphosphate tripolyhydrolase
gtp_c + h2o_c --> gsn_c + pppi_c
{}
gtp_c C10H12N5O14P3
h2o_c H2O
gsn_c C10H13N5O5
pppi_c HO10P3

Purine-nucleoside phosphorylase (Guanosine)
gsn_c + pi_c <=> gua_c + r1p_c
{'C': 5.0, 'H': 4.0, 'N': 5.0, 'O': 1.0}
gsn_c C10H13N5O5
pi_c HPO4
gua_c C5H4N5O;C5H5N5O
r1p_c C5H9O8P



In [158]:
get_metabolite_info(sorted_multiple_formula_metabolites[53])

fdxrd_c Reduced ferredoxin XH2;Fe8S8XH2;Fe2S2X
Ferredoxin---NADP+ reductase
2.0 fdxrd_c + h_c + nadp_c <=> 2.0 fdxo_2_2_c + nadph_c
{'H': -8.0, 'X': -4.0, 'Fe': -16.0, 'S': -16.0}
h_c H
nadp_c C21H25N7O17P3
fdxrd_c XH2;Fe8S8XH2;Fe2S2X
nadph_c C21H26N7O17P3
fdxo_2_2_c Fe2S2X

Ferredoxin-nitrate reductase
2.0 fdxrd_c + 2.0 h_c + no3_c --> 2.0 fdxo_2_2_c + h2o_c + no2_c
{'H': -8.0, 'X': -4.0, 'Fe': -16.0, 'S': -16.0}
h_c H
no3_c NO3
fdxrd_c XH2;Fe8S8XH2;Fe2S2X
h2o_c H2O
no2_c NO2
fdxo_2_2_c Fe2S2X

2-oxoglutarate synthase (rev)
akg_c + coa_c + fdxox_c + h_c --> co2_c + fdxrd_c + succoa_c
{'H': 2.0, 'S': 2.0, 'X': 1.0, 'Fe': 2.0}
akg_c C5H4O5
coa_c C21H32N7O16P3S
h_c H
fdxox_c X;Fe8S8X
co2_c CO2
succoa_c C25H35N7O19P3S
fdxrd_c XH2;Fe8S8XH2;Fe2S2X

FRDO
fdxrd_c + nadp_c <=> fdxox_c + h_c + nadph_c
{'H': -2.0, 'X': -1.0, 'Fe': -2.0, 'S': -2.0}
nadp_c C21H25N7O17P3
fdxrd_c XH2;Fe8S8XH2;Fe2S2X
h_c H
nadph_c C21H26N7O17P3
fdxox_c X;Fe8S8X

L-glutamate ferredoxin oxidoreductase
akg_c + 2.0 fdxrd

In [159]:
get_metabolite_info(sorted_multiple_formula_metabolites[54])

taur_c Taurine C2H7NO3S;C2H6NO3S
Sulfinoalanine decarboxylase
Lcyst_c --> co2_c + taur_c
{'C': 2.0, 'H': 7.0, 'N': 1.0, 'O': 3.0, 'S': 1.0}
Lcyst_c C3H6NO5S
co2_c CO2
taur_c C2H7NO3S;C2H6NO3S

Taurine dioxygenase
akg_c + o2_c + taur_c --> aacald_c + co2_c + h_c + so3_c + succ_c
{'C': -2.0, 'H': -5.0, 'N': -1.0}
akg_c C5H4O5
o2_c O2
taur_c C2H7NO3S;C2H6NO3S
aacald_c C2H6NO
co2_c CO2
h_c H
so3_c O3S;HSO3
succ_c C4H4O4

FMN dependent monooxygenase  taurine 
fmnRD_c + o2_c + taur_c --> amacald_c + fmn_c + h2o_c + h_c + so3_c
{'O': 9.0, 'C': 15.0, 'H': 12.0, 'N': 3.0, 'P': 1.0}
o2_c O2
taur_c C2H7NO3S;C2H6NO3S
fmnRD_c C17H21N4O9P
fmn_c C17H18N4O9P;C17H19N4O9P
h_c H
h2o_c H2O
so3_c O3S;HSO3
amacald_c C2H5NO

Taurine transport via ABC system
atp_c + h2o_c + taur_e --> adp_c + h_c + pi_c + taur_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
taur_e C2H7NO3S;C2H6NO3S
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
taur_c C2H7NO3S;C2H6NO3S

FDMOtau
fmnh2_c + o2_c + taur_c --> aacald_c + fmn_c + h2o_c + h_c + so3_c
{'C':

In [160]:
get_metabolite_info(sorted_multiple_formula_metabolites[55])

gua_c Guanine C5H4N5O;C5H5N5O
Purine-nucleoside phosphorylase (Deoxyguanosine)
dgsn_c + pi_c <=> 2dr1p_c + gua_c
{'C': 5.0, 'H': 4.0, 'N': 5.0, 'O': 1.0}
dgsn_c C10H13N5O4
pi_c HPO4
2dr1p_c C5H9O7P
gua_c C5H4N5O;C5H5N5O

Guanine deaminase
gua_c + h2o_c + h_c --> nh4_c + xan_c
{'C': -5.0, 'H': -4.0, 'N': -5.0, 'O': -1.0}
gua_c C5H4N5O;C5H5N5O
h_c H
h2o_c H2O
nh4_c H4N
xan_c C5H4N4O2

Gnnuc
gsn_c + h2o_c --> gua_c + rib__D_c
{'C': 5.0, 'H': 4.0, 'N': 5.0, 'O': 1.0}
gsn_c C10H13N5O5
h2o_c H2O
gua_c C5H4N5O;C5H5N5O
rib__D_c C5H10O5

Purine-nucleoside phosphorylase (Guanosine)
gsn_c + pi_c <=> gua_c + r1p_c
{'C': 5.0, 'H': 4.0, 'N': 5.0, 'O': 1.0}
gsn_c C10H13N5O5
pi_c HPO4
gua_c C5H4N5O;C5H5N5O
r1p_c C5H9O8P

Guanine reversible transport via proton symport
gua_e + h_e --> gua_c + h_c
{}
gua_e C5H4N5O;C5H5N5O
h_e H
gua_c C5H4N5O;C5H5N5O
h_c H



In [161]:
model.metabolites.get_by_id('gua_c').formula = 'C5H5N5O'
model.metabolites.get_by_id('gua_e').formula = 'C5H5N5O'

In [162]:
get_metabolite_info(sorted_multiple_formula_metabolites[55])

gua_c Guanine C5H5N5O
Purine-nucleoside phosphorylase (Deoxyguanosine)
dgsn_c + pi_c <=> 2dr1p_c + gua_c
{}
dgsn_c C10H13N5O4
pi_c HPO4
2dr1p_c C5H9O7P
gua_c C5H5N5O

Guanine deaminase
gua_c + h2o_c + h_c --> nh4_c + xan_c
{}
gua_c C5H5N5O
h_c H
h2o_c H2O
nh4_c H4N
xan_c C5H4N4O2

Gnnuc
gsn_c + h2o_c --> gua_c + rib__D_c
{}
gsn_c C10H13N5O5
h2o_c H2O
gua_c C5H5N5O
rib__D_c C5H10O5

Purine-nucleoside phosphorylase (Guanosine)
gsn_c + pi_c <=> gua_c + r1p_c
{}
gsn_c C10H13N5O5
pi_c HPO4
gua_c C5H5N5O
r1p_c C5H9O8P

Guanine reversible transport via proton symport
gua_e + h_e --> gua_c + h_c
{}
gua_e C5H5N5O
h_e H
gua_c C5H5N5O
h_c H



In [163]:
get_metabolite_info(sorted_multiple_formula_metabolites[56])

ribflv_c Riboflavin C17H19N4O6;C17H20N4O6
Flavin reductase
h_c + nadph_c + ribflv_c --> nadp_c + rbflvrd_c
{'H': -19.0, 'C': -17.0, 'N': -4.0, 'O': -6.0}
h_c H
nadph_c C21H26N7O17P3
ribflv_c C17H19N4O6;C17H20N4O6
nadp_c C21H25N7O17P3
rbflvrd_c C17H22N4O6

Biomass reaction
0.000216499790822049 10fthf_c + 0.498715520392769 ala__L_c + 0.000216499790822049 amet_c + 0.287169982631549 arg__L_c + 0.234028507069996 asn__L_c + 0.234028507069996 asp__L_c + 52.5471506268106 atp_c + 0.00505327987098101 ca2_c + 0.00505327987098101 cl_c + 0.000559210222033633 coa_c + 9.70851079919502e-05 cobalt2_c + 0.129616385977893 ctp_c + 0.000688333415662927 cu2_c + 0.088910541899028 cys__L_c + 0.0254032893571737 datp_c + 0.0262294836261852 dctp_c + 0.0262294836261852 dgtp_c + 0.0254032893571737 dttp_c + 0.000216499790822049 fad_c + 0.00651926500165946 fe2_c + 0.00758040523201147 fe3_c + 0.255489170191616 gln__L_c + 0.255489170191616 glu__L_c + 0.594780263899724 gly_c + 0.000970851079919502 gtca1_45_BS_c + 0.000

In [164]:
model.metabolites.get_by_id('ribflv_c').formula = 'C17H20N4O6'

In [165]:
get_metabolite_info(sorted_multiple_formula_metabolites[56])

ribflv_c Riboflavin C17H20N4O6
Flavin reductase
h_c + nadph_c + ribflv_c --> nadp_c + rbflvrd_c
{}
h_c H
nadph_c C21H26N7O17P3
ribflv_c C17H20N4O6
nadp_c C21H25N7O17P3
rbflvrd_c C17H22N4O6

Biomass reaction
0.000216499790822049 10fthf_c + 0.498715520392769 ala__L_c + 0.000216499790822049 amet_c + 0.287169982631549 arg__L_c + 0.234028507069996 asn__L_c + 0.234028507069996 asp__L_c + 52.5471506268106 atp_c + 0.00505327987098101 ca2_c + 0.00505327987098101 cl_c + 0.000559210222033633 coa_c + 9.70851079919502e-05 cobalt2_c + 0.129616385977893 ctp_c + 0.000688333415662927 cu2_c + 0.088910541899028 cys__L_c + 0.0254032893571737 datp_c + 0.0262294836261852 dctp_c + 0.0262294836261852 dgtp_c + 0.0254032893571737 dttp_c + 0.000216499790822049 fad_c + 0.00651926500165946 fe2_c + 0.00758040523201147 fe3_c + 0.255489170191616 gln__L_c + 0.255489170191616 glu__L_c + 0.594780263899724 gly_c + 0.000970851079919502 gtca1_45_BS_c + 0.000970851079919502 gtca2_45_BS_c + 0.000970851079919502 gtca3_45_BS_c

In [166]:
get_metabolite_info(sorted_multiple_formula_metabolites[57])

gthox_c Oxidized glutathione C20H30N6O12S2;C20H28N6O12S2
Glutaredoxin reductase
grxox_c + 2.0 gthrd_c --> grxrd_c + gthox_c
{'H': -2.0}
grxox_c X
gthrd_c C10H15N3O6S;C10H16N3O6S
grxrd_c XH2
gthox_c C20H30N6O12S2;C20H28N6O12S2

S-sulfo-L-cysteine sulfite lyase 
2.0 gthrd_c + scys__L_c --> cys__L_c + gthox_c + h_c + so3_c
{'H': -1.0, 'O': 3.0, 'S': 1.0}
gthrd_c C10H15N3O6S;C10H16N3O6S
scys__L_c C3H6NO5S2
cys__L_c C3H7NO2S
gthox_c C20H30N6O12S2;C20H28N6O12S2
h_c H
so3_c O3S;HSO3

MYCTR2
gthox_c + 2.0 msh_c <=> 2.0 gthrd_c + mssg_c
{'H': 2.0}
gthox_c C20H30N6O12S2;C20H28N6O12S2
msh_c C17H30N2O12S
gthrd_c C10H15N3O6S;C10H16N3O6S
mssg_c C34H58N4O24S2

Glutathione oxidoreductase
gthox_c + h_c + nadph_c <=> 2.0 gthrd_c + nadp_c
{'H': 2.0}
gthox_c C20H30N6O12S2;C20H28N6O12S2
h_c H
nadph_c C21H26N7O17P3
gthrd_c C10H15N3O6S;C10H16N3O6S
nadp_c C21H25N7O17P3

Arsenate reductase
aso4_c + 2.0 gthrd_c --> aso3_c + gthox_c + h2o_c
{'H': -2.0}
aso4_c AsO4
gthrd_c C10H15N3O6S;C10H16N3O6S
aso3_c AsO3
gtho

In [167]:
get_metabolite_info(sorted_multiple_formula_metabolites[58])

23dhbzs_c 2,3-dihydroxybenzoylserine C10H10NO6;C10H11NO6
2,3-dihydroxybenzoylserine hydrolase
23dhbzs_c + h2o_c --> 23dhb_c + ser__L_c
{'C': -10.0, 'H': -11.0, 'N': -1.0, 'O': -6.0}
23dhbzs_c C10H10NO6;C10H11NO6
h2o_c H2O
23dhb_c C7H5O4
ser__L_c C3H7NO3

Enterochelin Esterase
enter_c + 3.0 h2o_c --> 3.0 23dhbzs_c + 3.0 h_c
{'C': 30.0, 'H': 33.0, 'N': 3.0, 'O': 18.0}
enter_c C30H27N3O15
h2o_c H2O
23dhbzs_c C10H10NO6;C10H11NO6
h_c H

Enterochelin Esterase (Fe containing)
feenter_c + 3.0 h2o_c --> 3.0 23dhbzs_c + fe3_c + 3.0 h_c
{'C': 30.0, 'H': 33.0, 'N': 3.0, 'O': 18.0}
feenter_c C30H27FeN3O15
h2o_c H2O
23dhbzs_c C10H10NO6;C10H11NO6
fe3_c Fe
h_c H

23DHBS polymer hydrolase
23dhbzs3_c + h2o_c --> 23dhbzs2_c + 23dhbzs_c
{'H': 10.0, 'O': 6.0, 'C': 10.0, 'N': 1.0}
h2o_c H2O
23dhbzs3_c C30H29N3O16
23dhbzs_c C10H10NO6;C10H11NO6
23dhbzs2_c C20H20N2O11

Release of Fe(III) from ferric 2,3-dihydroxybenzoylserine
fe3dhbzs_c --> 23dhbzs_c + fe3_c
{'Fe': -1.0}
fe3dhbzs_c C10FeH11NO6;C10H10NO6Fe
23dh

# Conflict

In [169]:
get_metabolite_info(sorted_multiple_formula_metabolites[59])

urea_c Urea CH4N2O;CH5N2O
Urease
h2o_c + 2.0 h_c + urea_c --> co2_c + 2.0 nh4_c
{'H': -5.0, 'O': -1.0, 'C': -1.0, 'N': -2.0}
h_c H
h2o_c H2O
urea_c CH4N2O;CH5N2O
co2_c CO2
nh4_c H4N

Urea transport via facilitate diffusion
urea_e <=> urea_c
{}
urea_e CH4N2O;CH5N2O
urea_c CH4N2O;CH5N2O

Arginase
arg__L_c + h2o_c --> orn__L_c + urea_c
{'C': 1.0, 'H': 5.0, 'N': 2.0, 'O': 1.0}
arg__L_c C6H15N4O2
h2o_c H2O
urea_c CH4N2O;CH5N2O
orn__L_c C5H13N2O2

Allantoate amidinohydrolase  reversible
alltt_c + h2o_c <=> urdglyc_c + urea_c
{'C': 1.0, 'H': 5.0, 'N': 2.0, 'O': 1.0}
alltt_c C4H7N4O4
h2o_c H2O
urdglyc_c C3H5N2O4
urea_c CH4N2O;CH5N2O

Urea transport via ABC system (periplasm)
atp_c + h2o_c + urea_p --> adp_c + h_c + pi_c + urea_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
urea_p CH4N2O;CH5N2O
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
urea_c CH4N2O;CH5N2O

Urea transport via facilitate diffusion (periplasm)
urea_p <=> urea_c
{}
urea_p CH4N2O;CH5N2O
urea_c CH4N2O;CH5N2O

GUDBUTNAH
4gudbutn_c + h2o_c --> 4abut_c 

In [172]:
model.metabolites.get_by_id('urea_c').formula = 'CH4N2O'
model.metabolites.get_by_id('urea_e').formula = 'CH4N2O'
model.metabolites.get_by_id('urea_p').formula = 'CH4N2O'

In [173]:
get_metabolite_info(sorted_multiple_formula_metabolites[59])

urea_c Urea CH4N2O
Urease
h2o_c + 2.0 h_c + urea_c --> co2_c + 2.0 nh4_c
{}
h_c H
h2o_c H2O
urea_c CH4N2O
co2_c CO2
nh4_c H4N

Urea transport via facilitate diffusion
urea_e <=> urea_c
{}
urea_e CH4N2O
urea_c CH4N2O

Arginase
arg__L_c + h2o_c --> orn__L_c + urea_c
{}
arg__L_c C6H15N4O2
h2o_c H2O
urea_c CH4N2O
orn__L_c C5H13N2O2

Allantoate amidinohydrolase  reversible
alltt_c + h2o_c <=> urdglyc_c + urea_c
{}
alltt_c C4H7N4O4
h2o_c H2O
urdglyc_c C3H5N2O4
urea_c CH4N2O

Urea transport via ABC system (periplasm)
atp_c + h2o_c + urea_p --> adp_c + h_c + pi_c + urea_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
urea_p CH4N2O
adp_c C10H12N5O10P2
h_c H
pi_c HPO4
urea_c CH4N2O

Urea transport via facilitate diffusion (periplasm)
urea_p <=> urea_c
{}
urea_p CH4N2O
urea_c CH4N2O

GUDBUTNAH
4gudbutn_c + h2o_c --> 4abut_c + urea_c
{}
h2o_c H2O
4gudbutn_c C5H11N3O2
4abut_c C4H9NO2
urea_c CH4N2O



In [174]:
get_metabolite_info(sorted_multiple_formula_metabolites[60])

pydx5p_c Pyridoxal 5'-phosphate C8H8NO6P;C8H7NO6P
Pyridoxine 5'-phosphate oxidase
o2_c + pdx5p_c --> h2o2_c + pydx5p_c
{'O': 6.0, 'C': 8.0, 'H': 7.0, 'N': 1.0, 'P': 1.0}
o2_c O2
pdx5p_c C8H10NO6P
h2o2_c H2O2
pydx5p_c C8H8NO6P;C8H7NO6P

Pyridoxal-5-phosphate synthase
g3p_c + gln__L_c + r5p_c --> glu__L_c + 3.0 h2o_c + h_c + pi_c + pydx5p_c
{'C': 8.0, 'H': 7.0, 'O': 6.0, 'P': 1.0, 'N': 1.0}
g3p_c C3H5O6P
gln__L_c C5H10N2O3
r5p_c C5H9O8P
glu__L_c C5H8NO4
h_c H
h2o_c H2O
pi_c HPO4
pydx5p_c C8H8NO6P;C8H7NO6P

Pyridoxamine 5'-phosphate oxidase
h2o_c + o2_c + pyam5p_c --> h2o2_c + nh4_c + pydx5p_c
{'H': 7.0, 'O': 6.0, 'C': 8.0, 'N': 1.0, 'P': 1.0}
h2o_c H2O
o2_c O2
pyam5p_c C8H12N2O5P
h2o2_c H2O2
nh4_c H4N
pydx5p_c C8H8NO6P;C8H7NO6P

Alanine transaminase
ala__L_c + pydx5p_c --> pyam5p_c + pyr_c
{'C': -8.0, 'H': -7.0, 'N': -1.0, 'O': -6.0, 'P': -1.0}
ala__L_c C3H7NO2
pydx5p_c C8H8NO6P;C8H7NO6P
pyam5p_c C8H12N2O5P
pyr_c C3H3O3

Biomass reaction
0.000216499790822049 10fthf_c + 0.498715520392769 

In [175]:
model.metabolites.get_by_id('pydx5p_c').formula = 'C8H8NO6P'

In [176]:
get_metabolite_info(sorted_multiple_formula_metabolites[60])

pydx5p_c Pyridoxal 5'-phosphate C8H8NO6P
Pyridoxine 5'-phosphate oxidase
o2_c + pdx5p_c --> h2o2_c + pydx5p_c
{}
o2_c O2
pdx5p_c C8H10NO6P
h2o2_c H2O2
pydx5p_c C8H8NO6P

Pyridoxal-5-phosphate synthase
g3p_c + gln__L_c + r5p_c --> glu__L_c + 3.0 h2o_c + h_c + pi_c + pydx5p_c
{}
g3p_c C3H5O6P
gln__L_c C5H10N2O3
r5p_c C5H9O8P
glu__L_c C5H8NO4
h_c H
h2o_c H2O
pi_c HPO4
pydx5p_c C8H8NO6P

Pyridoxamine 5'-phosphate oxidase
h2o_c + o2_c + pyam5p_c --> h2o2_c + nh4_c + pydx5p_c
{}
h2o_c H2O
o2_c O2
pyam5p_c C8H12N2O5P
h2o2_c H2O2
nh4_c H4N
pydx5p_c C8H8NO6P

Alanine transaminase
ala__L_c + pydx5p_c --> pyam5p_c + pyr_c
{}
ala__L_c C3H7NO2
pydx5p_c C8H8NO6P
pyam5p_c C8H12N2O5P
pyr_c C3H3O3

Biomass reaction
0.000216499790822049 10fthf_c + 0.498715520392769 ala__L_c + 0.000216499790822049 amet_c + 0.287169982631549 arg__L_c + 0.234028507069996 asn__L_c + 0.234028507069996 asp__L_c + 52.5471506268106 atp_c + 0.00505327987098101 ca2_c + 0.00505327987098101 cl_c + 0.000559210222033633 coa_c + 9.708

# Conflict

In [179]:
get_metabolite_info(sorted_multiple_formula_metabolites[62])

trdox_c Oxidized thioredoxin X;C6H7NO2S2R2
L-methionine-S-oxide reductase
metsox_S__L_c + trdrd_c --> h2o_c + met__L_c + trdox_c
{'H': -2.0}
metsox_S__L_c C5H11NO3S
trdrd_c XH2;C6H9NO2S2R2
h2o_c H2O
met__L_c C5H11NO2S
trdox_c X;C6H7NO2S2R2

L-methionine-R-sulfoxide reductase
metsox_R__L_c + trdrd_c --> h2o_c + met__L_c + trdox_c
{'H': -2.0}
metsox_R__L_c C5H11NO3S
trdrd_c XH2;C6H9NO2S2R2
h2o_c H2O
met__L_c C5H11NO2S
trdox_c X;C6H7NO2S2R2

Cysteine synthase  Thiosulfate 
acser_c + trdrd_c + tsul_c <=> ac_c + cys__L_c + h_c + so3_c + trdox_c
{'H': -2.0, 'S': -1.0}
acser_c C5H9NO4
trdrd_c XH2;C6H9NO2S2R2
tsul_c HS2O3;O3S2
ac_c C2H3O2
cys__L_c C3H7NO2S
h_c H
so3_c O3S;HSO3
trdox_c X;C6H7NO2S2R2

L methionine S oxide reductase  trdrd 
metox_c + trdrd_c --> h2o_c + met__L_c + trdox_c
{'H': -2.0}
trdrd_c XH2;C6H9NO2S2R2
metox_c C5H11NO3S
h2o_c H2O
met__L_c C5H11NO2S
trdox_c X;C6H7NO2S2R2

Ribonucleoside-diphosphate reductase (UDP)
trdrd_c + udp_c --> dudp_c + h2o_c + trdox_c
{'H': -2.0}
trdrd

In [180]:
get_metabolite_info(sorted_multiple_formula_metabolites[63])

trdrd_c Reduced thioredoxin XH2;C6H9NO2S2R2
L-methionine-S-oxide reductase
metsox_S__L_c + trdrd_c --> h2o_c + met__L_c + trdox_c
{'H': -2.0}
metsox_S__L_c C5H11NO3S
trdrd_c XH2;C6H9NO2S2R2
h2o_c H2O
met__L_c C5H11NO2S
trdox_c X;C6H7NO2S2R2

L-methionine-R-sulfoxide reductase
metsox_R__L_c + trdrd_c --> h2o_c + met__L_c + trdox_c
{'H': -2.0}
metsox_R__L_c C5H11NO3S
trdrd_c XH2;C6H9NO2S2R2
h2o_c H2O
met__L_c C5H11NO2S
trdox_c X;C6H7NO2S2R2

Cysteine synthase  Thiosulfate 
acser_c + trdrd_c + tsul_c <=> ac_c + cys__L_c + h_c + so3_c + trdox_c
{'H': -2.0, 'S': -1.0}
acser_c C5H9NO4
trdrd_c XH2;C6H9NO2S2R2
tsul_c HS2O3;O3S2
ac_c C2H3O2
cys__L_c C3H7NO2S
h_c H
so3_c O3S;HSO3
trdox_c X;C6H7NO2S2R2

L methionine S oxide reductase  trdrd 
metox_c + trdrd_c --> h2o_c + met__L_c + trdox_c
{'H': -2.0}
trdrd_c XH2;C6H9NO2S2R2
metox_c C5H11NO3S
h2o_c H2O
met__L_c C5H11NO2S
trdox_c X;C6H7NO2S2R2

Ribonucleoside-diphosphate reductase (UDP)
trdrd_c + udp_c --> dudp_c + h2o_c + trdox_c
{'H': -2.0}
trdr

In [181]:
get_metabolite_info(sorted_multiple_formula_metabolites[64])

gthrd_c Reduced glutathione C10H15N3O6S;C10H16N3O6S
Formaldehyde glutathione ligase (spontaneous)
fald_c + gthrd_c <=> hmgth_c
{'C': -10.0, 'H': -15.0, 'O': -6.0, 'N': -3.0, 'S': -1.0}
fald_c CH2O
gthrd_c C10H15N3O6S;C10H16N3O6S
hmgth_c C11H18N3O7S

G glutamyltransferase
ala__L_c + gthrd_c --> cgly_c + gluala_c
{'C': -10.0, 'H': -15.0, 'N': -3.0, 'O': -6.0, 'S': -1.0}
ala__L_c C3H7NO2
gthrd_c C10H15N3O6S;C10H16N3O6S
cgly_c C5H10N2O3S
gluala_c C8H13N2O5

Glutathione oxidoreductase
gthox_c + h_c + nadph_c <=> 2.0 gthrd_c + nadp_c
{'H': 2.0}
gthox_c C20H30N6O12S2;C20H28N6O12S2
h_c H
nadph_c C21H26N7O17P3
gthrd_c C10H15N3O6S;C10H16N3O6S
nadp_c C21H25N7O17P3

Reduced glutathione via ABC system (periplasm)
atp_c + gthrd_p + h2o_c --> adp_c + gthrd_c + h_c + pi_c
{}
atp_c C10H12N5O13P3
h2o_c H2O
gthrd_p C10H15N3O6S;C10H16N3O6S
adp_c C10H12N5O10P2
gthrd_c C10H15N3O6S;C10H16N3O6S
h_c H
pi_c HPO4

MYCTR2
gthox_c + 2.0 msh_c <=> 2.0 gthrd_c + mssg_c
{'H': 2.0}
gthox_c C20H30N6O12S2;C20H28N6O12S2


In [182]:
get_metabolite_info(sorted_multiple_formula_metabolites[65])

fmn_c FMN C17H18N4O9P;C17H19N4O9P
FMN reductase  NADPH dependent 
fmn_c + h_c + nadph_c --> fmnRD_c + nadp_c
{'C': -17.0, 'H': -18.0, 'N': -4.0, 'O': -9.0, 'P': -1.0}
fmn_c C17H18N4O9P;C17H19N4O9P
h_c H
nadph_c C21H26N7O17P3
nadp_c C21H25N7O17P3
fmnRD_c C17H21N4O9P

FMNH2-dependent monooxygenase
fmnh2_c + isetac_c + o2_c --> fmn_c + gcald_c + h2o_c + h_c + so3_c
{'C': 17.0, 'H': 19.0, 'N': 4.0, 'O': 12.0, 'P': 1.0, 'S': 1.0}
fmnh2_c C17H21N4O9P
isetac_c C2H5O4S
o2_c O2
fmn_c C17H18N4O9P;C17H19N4O9P
gcald_c C2H4O2
h_c H
h2o_c H2O
so3_c O3S;HSO3

FDMOtau
fmnh2_c + o2_c + taur_c --> aacald_c + fmn_c + h2o_c + h_c + so3_c
{'C': 15.0, 'H': 13.0, 'N': 3.0, 'O': 9.0, 'P': 1.0}
fmnh2_c C17H21N4O9P
o2_c O2
taur_c C2H7NO3S;C2H6NO3S
aacald_c C2H6NO
fmn_c C17H18N4O9P;C17H19N4O9P
h_c H
h2o_c H2O
so3_c O3S;HSO3

FMN reductase
fmn_c + h_c + nadh_c --> fmnh2_c + nad_c
{'C': -17.0, 'H': -18.0, 'N': -4.0, 'O': -9.0, 'P': -1.0}
fmn_c C17H18N4O9P;C17H19N4O9P
h_c H
nadh_c C21H27N7O14P2
fmnh2_c C17H21N4O9P


In [183]:
get_metabolite_info(sorted_multiple_formula_metabolites[66])

so3_c Sulfite O3S;HSO3
Taurine dioxygenase
akg_c + o2_c + taur_c --> aacald_c + co2_c + h_c + so3_c + succ_c
{'C': -2.0, 'H': -5.0, 'N': -1.0}
akg_c C5H4O5
o2_c O2
taur_c C2H7NO3S;C2H6NO3S
aacald_c C2H6NO
co2_c CO2
h_c H
so3_c O3S;HSO3
succ_c C4H4O4

FMNH2-dependent monooxygenase
fmnh2_c + isetac_c + o2_c --> fmn_c + gcald_c + h2o_c + h_c + so3_c
{'C': 17.0, 'H': 19.0, 'N': 4.0, 'O': 12.0, 'P': 1.0, 'S': 1.0}
fmnh2_c C17H21N4O9P
isetac_c C2H5O4S
o2_c O2
fmn_c C17H18N4O9P;C17H19N4O9P
gcald_c C2H4O2
h_c H
h2o_c H2O
so3_c O3S;HSO3

Cysteine synthase  Thiosulfate 
acser_c + trdrd_c + tsul_c <=> ac_c + cys__L_c + h_c + so3_c + trdox_c
{'H': -2.0, 'S': -1.0}
acser_c C5H9NO4
trdrd_c XH2;C6H9NO2S2R2
tsul_c HS2O3;O3S2
ac_c C2H3O2
cys__L_c C3H7NO2S
h_c H
so3_c O3S;HSO3
trdox_c X;C6H7NO2S2R2

FDMOtau
fmnh2_c + o2_c + taur_c --> aacald_c + fmn_c + h2o_c + h_c + so3_c
{'C': 15.0, 'H': 13.0, 'N': 3.0, 'O': 9.0, 'P': 1.0}
fmnh2_c C17H21N4O9P
o2_c O2
taur_c C2H7NO3S;C2H6NO3S
aacald_c C2H6NO
fmn_c C17H

In [184]:
get_metabolite_info(sorted_multiple_formula_metabolites[67])

malcoa_c Malonyl-CoA C24H34N7O19P3S;C24H33N7O19P3S
Fatty acid synthase  n C120 
dca_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + ddca_c + h2o_c + 2.0 nadp_c
{'C': -24.0, 'H': -34.0, 'O': -19.0, 'N': -7.0, 'P': -3.0, 'S': -1.0}
dca_c C10H19O2
h_c H
malcoa_c C24H34N7O19P3S;C24H33N7O19P3S
nadph_c C21H26N7O17P3
co2_c CO2
coa_c C21H32N7O16P3S
ddca_c C12H23O2
h2o_c H2O
nadp_c C21H25N7O17P3

Fatty acid synthase  n C161 
4.0 h_c + malcoa_c + 3.0 nadph_c + o2_c + ttdca_c --> co2_c + coa_c + 3.0 h2o_c + hdcea_c + 3.0 nadp_c
{'H': -34.0, 'C': -24.0, 'N': -7.0, 'O': -19.0, 'P': -3.0, 'S': -1.0}
h_c H
malcoa_c C24H34N7O19P3S;C24H33N7O19P3S
nadph_c C21H26N7O17P3
o2_c O2
ttdca_c C14H27O2
co2_c CO2
coa_c C21H32N7O16P3S
h2o_c H2O
hdcea_c C16H29O2
nadp_c C21H25N7O17P3

B-ketoacyl synthetase (n-C14:0)
accoa_c + 17.0 h_c + 6.0 malcoa_c + 12.0 nadph_c --> 6.0 co2_c + 7.0 coa_c + 5.0 h2o_c + 12.0 nadp_c + ttdca_c
{'C': -144.0, 'H': -204.0, 'N': -42.0, 'O': -114.0, 'P': -18.0, 'S': -6.0}
accoa_c 

# Status Report

In [186]:
for i in range(1,5):
    num_formulas = [m for m in model.metabolites if len(m.formula.split(';')) == i]
    print(f'{len(num_formulas)} of {len(model.metabolites)} metabolites have {i} formula(s)')

1547 of 1581 metabolites have 1 formula(s)
32 of 1581 metabolites have 2 formula(s)
2 of 1581 metabolites have 3 formula(s)
0 of 1581 metabolites have 4 formula(s)


In [187]:
unbalanced = [r for r in model.reactions if should_be_balanced(r) and r.check_mass_balance() != {}]
unbalanced_but_okay = [r for r in model.reactions if not should_be_balanced(r) and r.check_mass_balance() != {}]
balanced = [r for r in model.reactions if r.check_mass_balance() == {}]

print(f'{len(unbalanced)} of the {len(model.reactions)} reactions in the model are wrongly unbalanced')
print(f'{len(unbalanced_but_okay)} of the {len(model.reactions)} reactions in the model are properly unbalanced')
print(f'{len(balanced)} of the {len(model.reactions)} reactions in the model are balanced')

120 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
2032 of the 2380 reactions in the model are balanced


In [188]:
unbalanced_multiple_formulas = [r for r in unbalanced if has_metabolite_with_multiple_formulas(r)]
unbalanced_but_okay_multiple_formulas = [r for r in unbalanced_but_okay if has_metabolite_with_multiple_formulas(r)]
balanced_multiple_formulas   = [r for r in   balanced if has_metabolite_with_multiple_formulas(r)]

print(f'{len(unbalanced_multiple_formulas)} of the {len(unbalanced)} improperly unbalanced reactions in the model have at least one metabolite with multiple formulas')
print(f'{len(unbalanced_but_okay_multiple_formulas)} of the {len(unbalanced_but_okay)} properly unbalanced reactions in the model have at least one metabolite with multiple formulas')
print(f'{len(balanced_multiple_formulas)} of the {len(balanced)} balanced reactions in the model have at least one metabolite with multiple formulas')

117 of the 120 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
5 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
11 of the 2032 balanced reactions in the model have at least one metabolite with multiple formulas


In [189]:
multiple_formula_metabolites = [m for m in model.metabolites if len(m.formula.split(';')) > 1]
sorted_multiple_formula_metabolites = sorted(multiple_formula_metabolites, key=lambda m: len(m.reactions))
    
sorted_multiple_formula_metabolites[0]

Metabolite identifier,fdxox_c
Name,Oxidized ferredoxin
Memory address,0x07f300bd03e90
Formula,X;Fe8S8X
Compartment,C_c
In 2 reaction(s),"OOR3r, FRDO"
